# Rice

## Standard KNN

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy.io import arff
import logging
from scipy import stats

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[logging.FileHandler('k1_rice_standard_knn.log'), logging.StreamHandler()])

data_path = 'rice/Rice_Cammeo_Osmancik.arff'
data, meta = arff.loadarff(data_path)
data_df = pd.DataFrame(data)

data_df['Class'] = data_df['Class'].apply(lambda x: x.decode('utf-8'))

label_encoder = LabelEncoder()
data_df['Class'] = label_encoder.fit_transform(data_df['Class'])

X = data_df.drop(columns=['Class'])
y = data_df['Class']

kf = KFold(n_splits=5, shuffle=True, random_state=42)

k_value = 1

accuracies = []
precisions = []
recalls = []
f1_scores = []
times = []
best_y_test = None
best_predictions = None

def evaluate_robustness(best_model, X_test, y_test, noise_levels=[0.1, 0.2, 0.3, 0.4, 0.5], n_bootstrap=100):
    results = []

    n_samples = len(y_test)

    original_pred = best_model.predict(X_test)
    original_acc = accuracy_score(y_test, original_pred)
    original_f1 = f1_score(y_test, original_pred, average='weighted')
    results.append(('No Noise', original_acc, original_f1))

    bootstrap_original_acc = []
    bootstrap_original_f1 = []

    for _ in range(n_bootstrap):
        indices = np.random.choice(n_samples, n_samples, replace=True)
        X_boot = X_test[indices]
        y_boot = y_test.iloc[indices] if hasattr(y_test, 'iloc') else y_test[indices]

        boot_pred = best_model.predict(X_boot)
        bootstrap_original_acc.append(accuracy_score(y_boot, boot_pred))
        bootstrap_original_f1.append(f1_score(y_boot, boot_pred, average='weighted'))

    p_values_acc = []
    p_values_f1 = []
    bootstrap_acc_all = []
    bootstrap_f1_all = []

    for noise in noise_levels:
        bootstrap_acc = []
        bootstrap_f1 = []

        for _ in range(n_bootstrap):
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_boot = X_test[indices].copy()
            y_boot = y_test.iloc[indices] if hasattr(y_test, 'iloc') else y_test[indices]

            noise_matrix = np.random.normal(0, noise, X_boot.shape)
            X_boot += noise_matrix

            noisy_pred = best_model.predict(X_boot)
            noisy_acc = accuracy_score(y_boot, noisy_pred)
            noisy_f1 = f1_score(y_boot, noisy_pred, average='weighted')

            bootstrap_acc.append(noisy_acc)
            bootstrap_f1.append(noisy_f1)

        mean_acc = np.mean(bootstrap_acc)
        mean_f1 = np.mean(bootstrap_f1)

        t_stat_acc, p_val_acc = stats.ttest_rel(bootstrap_original_acc, bootstrap_acc)
        t_stat_f1, p_val_f1 = stats.ttest_rel(bootstrap_original_f1, bootstrap_f1)

        results.append((f'Noise {noise:.2f}', mean_acc, mean_f1))
        p_values_acc.append(p_val_acc)
        p_values_f1.append(p_val_f1)
        bootstrap_acc_all.append(bootstrap_acc)
        bootstrap_f1_all.append(bootstrap_f1)

    plt.figure(figsize=(12, 7))
    noise_labels = [r[0] for r in results]
    acc_values = [r[1] for r in results]
    f1_values = [r[2] for r in results]

    x = np.arange(len(noise_labels))
    width = 0.35

    plt.subplot(1, 2, 1)
    bars_acc = plt.bar(x - width/2, acc_values, width, label='Accuracy')
    bars_f1 = plt.bar(x + width/2, f1_values, width, label='F1 Score')
    plt.xlabel('Noise Level')
    plt.ylabel('Score')
    plt.title('Model Robustness to Noise')
    plt.xticks(x, noise_labels, rotation=45)
    plt.legend()

    for i in range(len(p_values_acc)):
        if i > 0:
            if p_values_acc[i-1] < 0.001:
                plt.text(i - width/2, acc_values[i] - 0.02, '***', ha='center')
            elif p_values_acc[i-1] < 0.01:
                plt.text(i - width/2, acc_values[i] - 0.02, '**', ha='center')
            elif p_values_acc[i-1] < 0.05:
                plt.text(i - width/2, acc_values[i] - 0.02, '*', ha='center')

            if p_values_f1[i-1] < 0.001:
                plt.text(i + width/2, f1_values[i] - 0.02, '***', ha='center')
            elif p_values_f1[i-1] < 0.01:
                plt.text(i + width/2, f1_values[i] - 0.02, '**', ha='center')
            elif p_values_f1[i-1] < 0.05:
                plt.text(i + width/2, f1_values[i] - 0.02, '*', ha='center')

    plt.subplot(1, 2, 2)
    bars_acc = plt.bar(x - width/2, acc_values, width, label='Accuracy')
    bars_f1 = plt.bar(x + width/2, f1_values, width, label='F1 Score')
    plt.xlabel('Noise Level')
    plt.ylabel('Score')
    plt.title('Zoomed View (0.85-1.0 range) with p-values')
    plt.xticks(x, noise_labels, rotation=45)
    plt.ylim(0.85, 1.0)

    for i in range(len(p_values_acc)):
        if i > 0:
            if p_values_acc[i-1] < 0.001:
                plt.text(i - width/2, acc_values[i] + 0.005, '***', ha='center', fontsize=10)
            elif p_values_acc[i-1] < 0.01:
                plt.text(i - width/2, acc_values[i] + 0.005, '**', ha='center', fontsize=10)
            elif p_values_acc[i-1] < 0.05:
                plt.text(i - width/2, acc_values[i] + 0.005, '*', ha='center', fontsize=10)

            if p_values_f1[i-1] < 0.001:
                plt.text(i + width/2, f1_values[i] + 0.005, '***', ha='center', fontsize=10)
            elif p_values_f1[i-1] < 0.01:
                plt.text(i + width/2, f1_values[i] + 0.005, '**', ha='center', fontsize=10)
            elif p_values_f1[i-1] < 0.05:
                plt.text(i + width/2, f1_values[i] + 0.005, '*', ha='center', fontsize=10)

    for i, (acc, f1) in enumerate(zip(acc_values, f1_values)):
        plt.text(i - width/2, acc - 0.01, f'{acc:.7f}', ha='center', va='bottom', fontsize=8, rotation=90)
        plt.text(i + width/2, f1 - 0.01, f'{f1:.7f}', ha='center', va='bottom', fontsize=8, rotation=90)

    plt.figtext(0.5, 0.01, "* p<0.05, ** p<0.01, *** p<0.001", ha="center", fontsize=10)

    plt.tight_layout()
    plt.savefig(f'standard_knn_k{k_value}_noise_robustness.png')
    plt.close()

    plt.figure(figsize=(14, 8))

    plt.subplot(2, 1, 1)
    boxplot_data_acc = [bootstrap_original_acc] + bootstrap_acc_all
    plt.boxplot(boxplot_data_acc, labels=noise_labels, showfliers=False)
    plt.title('Distribution of Accuracy Scores Across Noise Levels')
    plt.ylabel('Accuracy')
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.subplot(2, 1, 2)
    boxplot_data_f1 = [bootstrap_original_f1] + bootstrap_f1_all
    plt.boxplot(boxplot_data_f1, labels=noise_labels, showfliers=False)
    plt.title('Distribution of F1 Scores Across Noise Levels')
    plt.ylabel('F1 Score')
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig(f'standard_knn_k{k_value}_noise_distribution.png')
    plt.close()

    logging.info("\n===== NOISE ROBUSTNESS RESULTS WITH P-VALUES =====")
    logging.info(f"{'Noise Level':<12} {'Accuracy':<12} {'F1 Score':<12} {'p-val (Acc)':<12} {'p-val (F1)':<12}")

    logging.info(f"{noise_labels[0]:<12} {acc_values[0]:.7f}      {f1_values[0]:.7f}      -            -")

    for i in range(1, len(noise_labels)):
        logging.info(f"{noise_labels[i]:<12} {acc_values[i]:.7f}      {f1_values[i]:.7f}      {p_values_acc[i-1]:.2e}     {p_values_f1[i-1]:.2e}")

    return results, p_values_acc, p_values_f1

def analyze_results():

    acc_mean, acc_std = np.mean(accuracies), np.std(accuracies)
    prec_mean, prec_std = np.mean(precisions), np.std(precisions)
    rec_mean, rec_std = np.mean(recalls), np.std(recalls)
    f1_mean, f1_std = np.mean(f1_scores), np.std(f1_scores)
    time_mean, time_std = np.mean(times), np.std(times)

    logging.info("\n===== RESULTS (Standard KNN) =====")
    logging.info(f"k value (n_neighbors): {k_value}")
    logging.info(f"Accuracy: {acc_mean:.7f} ± {acc_std:.7f}")
    logging.info(f"Precision: {prec_mean:.7f} ± {prec_std:.7f}")
    logging.info(f"Recall: {rec_mean:.7f} ± {rec_std:.7f}")
    logging.info(f"F1 Score: {f1_mean:.7f} ± {f1_std:.7f}")
    logging.info(f"Average execution time: {time_mean:.7f}s ± {time_std:.7f}s")

    plt.figure(figsize=(10, 6))
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    values = [acc_mean, prec_mean, rec_mean, f1_mean]
    errors = [acc_std, prec_std, rec_std, f1_std]

    bars = plt.bar(metrics, values, yerr=errors, capsize=10)
    plt.title(f'Performance Metrics with Error Bars (Standard KNN, n_neighbors={k_value})')
    plt.ylabel('Score')
    plt.ylim(0, 1.1)

    for bar, val, err in zip(bars, values, errors):
        plt.text(bar.get_x() + bar.get_width()/2., val + err + 0.02,
                f'{val:.7f}±{err:.7f}', ha='center', va='bottom', rotation=0)

    plt.tight_layout()
    plt.savefig(f'standard_knn_k{k_value}_performance_metrics.png')
    plt.close()

    if best_y_test is not None and best_predictions is not None:
        plt.figure(figsize=(8, 6))
        cm = confusion_matrix(best_y_test, best_predictions)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f'Confusion Matrix for Standard KNN (k={k_value})')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.savefig(f'standard_knn_k{k_value}_confusion_matrix.png')
        plt.close()

def main():
    global accuracies, precisions, recalls, f1_scores, times, best_y_test, best_predictions

    start_time = time.time()

    logging.info(f"Standard KNN test started (k={k_value})...")

    best_acc = -float('inf')

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        fold_start_time = time.time()

        knn = KNeighborsClassifier(n_neighbors=k_value)
        knn.fit(X_train_scaled, y_train)

        predictions = knn.predict(X_test_scaled)

        fold_end_time = time.time()
        exec_time = fold_end_time - fold_start_time

        accuracy = accuracy_score(y_test, predictions)
        precision = precision_score(y_test, predictions, average='weighted')
        recall = recall_score(y_test, predictions, average='weighted')
        f1 = f1_score(y_test, predictions, average='weighted')

        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
        times.append(exec_time)

        logging.info(f"Fold, Acc: {accuracy:.7f}, Prec: {precision:.7f}, Rec: {recall:.7f}, F1: {f1:.7f}, Time: {exec_time:.7f}s")

        if accuracy > best_acc:
            best_acc = accuracy
            best_y_test = y_test
            best_predictions = predictions

    total_time = time.time() - start_time
    logging.info(f"Test completed. Total time: {total_time:.2f} second")

    analyze_results()

    logging.info("Noise resistance test started...")

    best_fold_idx = np.argmax(accuracies)
    train_indices = list(kf.split(X))[best_fold_idx][0]
    test_indices = list(kf.split(X))[best_fold_idx][1]

    X_train = X.iloc[train_indices]
    X_test = X.iloc[test_indices]
    y_train = y[train_indices]
    y_test = y[test_indices]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    best_model = KNeighborsClassifier(n_neighbors=k_value)
    best_model.fit(X_train_scaled, y_train)

    noise_levels = [0.1, 0.2, 0.3, 0.4, 0.5]
    n_bootstrap = 100

    noise_results, p_values_acc, p_values_f1 = evaluate_robustness(
        best_model, X_test_scaled, y_test, noise_levels, n_bootstrap)

    logging.info("Code snippet completed...")

if __name__ == "__main__":
    main()

## 10% Train-data KNN

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy.io import arff
import logging
from scipy import stats

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[logging.FileHandler('k1_rice_one_tenth_train_data_knn.log'), logging.StreamHandler()])

data_path = 'rice/Rice_Cammeo_Osmancik.arff'
data, meta = arff.loadarff(data_path)
data_df = pd.DataFrame(data)

data_df['Class'] = data_df['Class'].apply(lambda x: x.decode('utf-8'))

label_encoder = LabelEncoder()
data_df['Class'] = label_encoder.fit_transform(data_df['Class'])
logging.info(f"Class Labels: {dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))}")


X = data_df.drop(columns=['Class'])
y = data_df['Class']

N_OUTER_SPLITS = 5
kf = KFold(n_splits=N_OUTER_SPLITS, shuffle=True, random_state=42)

k_value = 1
N_INNER_SPLITS = 10

all_individual_accuracies = []
all_individual_precisions = []
all_individual_recalls = []
all_individual_f1_scores = []
all_individual_times = []

all_inner_model_details = []


def evaluate_robustness(model_to_test, X_test, y_test, noise_levels=[0.1, 0.2, 0.3, 0.4, 0.5], n_bootstrap=100, model_desc="Test Model"):
    results = []
    logging.info(f"evaluate_robustness for {model_desc} - X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

    n_samples = len(y_test)
    if n_samples == 0:
        logging.warning(f"evaluate_robustness ({model_desc}): Test data is empty, bootstrap cannot be done.")
        for noise in noise_levels: results.append((f'Noise {noise:.2f}', np.nan, np.nan))
        return results, [np.nan]*len(noise_levels), [np.nan]*len(noise_levels)

    original_pred = model_to_test.predict(X_test)
    original_acc = accuracy_score(y_test, original_pred)
    original_f1 = f1_score(y_test, original_pred, average='weighted', zero_division=0)
    results.append(('No Noise', original_acc, original_f1))
    logging.info(f"evaluate_robustness ({model_desc}) - No Noise: Acc={original_acc:.7f}, F1={original_f1:.7f}")

    bootstrap_original_acc = []
    bootstrap_original_f1 = []
    for _ in range(n_bootstrap):
        indices = np.random.choice(n_samples, n_samples, replace=True)
        X_boot, y_boot = X_test[indices], (y_test.iloc[indices] if hasattr(y_test, 'iloc') else y_test[indices])
        boot_pred = model_to_test.predict(X_boot)
        bootstrap_original_acc.append(accuracy_score(y_boot, boot_pred))
        bootstrap_original_f1.append(f1_score(y_boot, boot_pred, average='weighted', zero_division=0))

    p_values_acc, p_values_f1, bootstrap_acc_all, bootstrap_f1_all = [], [], [], []

    for noise in noise_levels:
        bootstrap_acc, bootstrap_f1 = [], []
        for _ in range(n_bootstrap):
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_boot, y_boot = X_test[indices].copy(), (y_test.iloc[indices] if hasattr(y_test, 'iloc') else y_test[indices])
            noise_matrix = np.random.normal(0, noise, X_boot.shape)
            X_boot += noise_matrix
            noisy_pred = model_to_test.predict(X_boot)
            bootstrap_acc.append(accuracy_score(y_boot, noisy_pred))
            bootstrap_f1.append(f1_score(y_boot, noisy_pred, average='weighted', zero_division=0))

        mean_acc, mean_f1 = (np.mean(bootstrap_acc) if bootstrap_acc else np.nan), (np.mean(bootstrap_f1) if bootstrap_f1 else np.nan)
        p_val_acc, p_val_f1 = (stats.ttest_rel(bootstrap_original_acc, bootstrap_acc)[1] if bootstrap_original_acc and bootstrap_acc else np.nan), \
                              (stats.ttest_rel(bootstrap_original_f1, bootstrap_f1)[1] if bootstrap_original_f1 and bootstrap_f1 else np.nan)

        results.append((f'Noise {noise:.2f}', mean_acc, mean_f1))
        p_values_acc.append(p_val_acc); p_values_f1.append(p_val_f1)
        bootstrap_acc_all.append(bootstrap_acc); bootstrap_f1_all.append(bootstrap_f1)

    plt.figure(figsize=(13, 7))
    noise_labels = [r[0] for r in results]
    acc_values = [r[1] for r in results]
    f1_values_plot = [r[2] for r in results]
    x_indices = np.arange(len(noise_labels))
    width = 0.35

    plt.subplot(1, 2, 1)
    plt.bar(x_indices - width/2, acc_values, width, label='Accuracy')
    plt.bar(x_indices + width/2, f1_values_plot, width, label='F1 Score')
    plt.xlabel('Noise Level'); plt.ylabel('Score'); plt.title(f'Model Robustness to Noise ({model_desc})')
    plt.xticks(x_indices, noise_labels, rotation=45); plt.legend()
    for i in range(len(p_values_acc)):
        y_star_acc = acc_values[i+1] - 0.02 if not np.isnan(acc_values[i+1]) else 0
        y_star_f1 = f1_values_plot[i+1] - 0.02 if not np.isnan(f1_values_plot[i+1]) else 0
        if not np.isnan(p_values_acc[i]):
            if p_values_acc[i] < 0.001: plt.text(x_indices[i+1] - width/2, y_star_acc, '***', ha='center')
            elif p_values_acc[i] < 0.01: plt.text(x_indices[i+1] - width/2, y_star_acc, '**', ha='center')
            elif p_values_acc[i] < 0.05: plt.text(x_indices[i+1] - width/2, y_star_acc, '*', ha='center')
        if not np.isnan(p_values_f1[i]):
            if p_values_f1[i] < 0.001: plt.text(x_indices[i+1] + width/2, y_star_f1, '***', ha='center')
            elif p_values_f1[i] < 0.01: plt.text(x_indices[i+1] + width/2, y_star_f1, '**', ha='center')
            elif p_values_f1[i] < 0.05: plt.text(x_indices[i+1] + width/2, y_star_f1, '*', ha='center')

    plt.subplot(1, 2, 2)
    valid_scores = [s for s in acc_values + f1_values_plot if not np.isnan(s)]
    zoom_min_val = max(0.0, min(valid_scores) - 0.05) if valid_scores else 0.0
    zoom_max_val = min(1.0, max(valid_scores) + 0.05) if valid_scores else 1.0
    if zoom_min_val >= zoom_max_val: zoom_min_val, zoom_max_val = 0.0, 1.0

    plt.bar(x_indices - width/2, acc_values, width, label='Accuracy')
    plt.bar(x_indices + width/2, f1_values_plot, width, label='F1 Score')
    plt.xlabel('Noise Level'); plt.ylabel('Score'); plt.title(f'Zoomed View ({zoom_min_val:.2f}-{zoom_max_val:.2f})')
    plt.xticks(x_indices, noise_labels, rotation=45); plt.ylim(zoom_min_val, zoom_max_val)
    for i in range(len(p_values_acc)):
        if not np.isnan(acc_values[i+1]):
            y_text_acc = acc_values[i+1] + 0.005 if acc_values[i+1] < zoom_max_val - 0.01 else acc_values[i+1] - 0.01
            plt.text(x_indices[i+1] - width/2, acc_values[i+1] - (zoom_max_val-zoom_min_val)*0.05 if acc_values[i+1] > zoom_min_val + (zoom_max_val-zoom_min_val)*0.1 else acc_values[i+1] + (zoom_max_val-zoom_min_val)*0.02, f'{acc_values[i+1]:.3f}', ha='center', va='bottom' if acc_values[i+1] > zoom_min_val + (zoom_max_val-zoom_min_val)*0.1 else 'top', fontsize=7, rotation=90)
            if not np.isnan(p_values_acc[i]):
                if p_values_acc[i] < 0.001: plt.text(x_indices[i+1] - width/2, y_text_acc, '***', ha='center', fontsize=10)
                elif p_values_acc[i] < 0.01: plt.text(x_indices[i+1] - width/2, y_text_acc, '**', ha='center', fontsize=10)
                elif p_values_acc[i] < 0.05: plt.text(x_indices[i+1] - width/2, y_text_acc, '*', ha='center', fontsize=10)
        if not np.isnan(f1_values_plot[i+1]):
            y_text_f1 = f1_values_plot[i+1] + 0.005 if f1_values_plot[i+1] < zoom_max_val - 0.01 else f1_values_plot[i+1] - 0.01
            plt.text(x_indices[i+1] + width/2, f1_values_plot[i+1] - (zoom_max_val-zoom_min_val)*0.05 if f1_values_plot[i+1] > zoom_min_val + (zoom_max_val-zoom_min_val)*0.1 else f1_values_plot[i+1] + (zoom_max_val-zoom_min_val)*0.02, f'{f1_values_plot[i+1]:.3f}', ha='center', va='bottom' if f1_values_plot[i+1] > zoom_min_val + (zoom_max_val-zoom_min_val)*0.1 else 'top', fontsize=7, rotation=90)
            if not np.isnan(p_values_f1[i]):
                if p_values_f1[i] < 0.001: plt.text(x_indices[i+1] + width/2, y_text_f1, '***', ha='center', fontsize=10)
                elif p_values_f1[i] < 0.01: plt.text(x_indices[i+1] + width/2, y_text_f1, '**', ha='center', fontsize=10)
                elif p_values_f1[i] < 0.05: plt.text(x_indices[i+1] + width/2, y_text_f1, '*', ha='center', fontsize=10)

    plt.figtext(0.5, 0.01, "* p<0.05, ** p<0.01, *** p<0.001 (vs No Noise)", ha="center", fontsize=10)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.suptitle(f'Model Robustness ({model_desc}, k={k_value})', fontsize=14)
    plt.savefig(f'modified_knn_strategy3_k{k_value}_{model_desc.replace(" ","_")}_robustness.png')
    plt.close()

    plt.figure(figsize=(14, 8))
    boxplot_labels = ['No Noise'] + [f'Noise {n:.2f}' for n in noise_levels]
    plt.subplot(2, 1, 1);
    valid_boxplot_acc = [d for d in ([bootstrap_original_acc] + bootstrap_acc_all) if d]
    if valid_boxplot_acc: plt.boxplot(valid_boxplot_acc, labels=boxplot_labels[:len(valid_boxplot_acc)], showfliers=False)
    plt.title(f'Distribution of Accuracy Scores ({model_desc})'); plt.ylabel('Accuracy'); plt.grid(True, linestyle='--', alpha=0.7)

    plt.subplot(2, 1, 2);
    valid_boxplot_f1 = [d for d in ([bootstrap_original_f1] + bootstrap_f1_all) if d]
    if valid_boxplot_f1: plt.boxplot(valid_boxplot_f1, labels=boxplot_labels[:len(valid_boxplot_f1)], showfliers=False)
    plt.title(f'Distribution of F1 Scores ({model_desc})'); plt.ylabel('F1 Score'); plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(f'modified_knn_strategy3_k{k_value}_{model_desc.replace(" ","_")}_noise_distribution.png')
    plt.close()

    logging.info(f"\n===== NOISE ROBUSTNESS RESULTS ({model_desc}) =====")
    logging.info(f"{'Noise Level':<12} {'Mean Acc':<12} {'Mean F1':<12} {'p-val (Acc)':<12} {'p-val (F1)':<12}")
    acc_val_str = f"{acc_values[0]:.7f}" if not np.isnan(acc_values[0]) else "NaN"
    f1_val_str = f"{f1_values_plot[0]:.7f}" if not np.isnan(f1_values_plot[0]) else "NaN"
    logging.info(f"{noise_labels[0]:<12} {acc_val_str:<12} {f1_val_str:<12} -            -")
    for i in range(len(noise_levels)):
        p_acc_str = f"{p_values_acc[i]:.2e}" if not np.isnan(p_values_acc[i]) else "N/A"
        p_f1_str = f"{p_values_f1[i]:.2e}" if not np.isnan(p_values_f1[i]) else "N/A"
        acc_val_str = f"{acc_values[i+1]:.7f}" if not np.isnan(acc_values[i+1]) else "NaN"
        f1_val_str = f"{f1_values_plot[i+1]:.7f}" if not np.isnan(f1_values_plot[i+1]) else "NaN"
        logging.info(f"{noise_labels[i+1]:<12} {acc_val_str:<12} {f1_val_str:<12} {p_acc_str:<12} {p_f1_str:<12}")
    return results, p_values_acc, p_values_f1

def analyze_results(model_for_cm=None, X_cm=None, y_cm=None, model_desc="Test Model"):
    if not all_individual_accuracies:
        logging.warning("analyze_results: No model results found to calculate.")
        acc_mean, acc_std, prec_mean, prec_std, rec_mean, rec_std, f1_mean, f1_std, time_mean, time_std = [np.nan]*10
    else:
        acc_mean, acc_std = np.mean(all_individual_accuracies), np.std(all_individual_accuracies)
        prec_mean, prec_std = np.mean(all_individual_precisions), np.std(all_individual_precisions)
        rec_mean, rec_std = np.mean(all_individual_recalls), np.std(all_individual_recalls)
        f1_mean, f1_std = np.mean(all_individual_f1_scores), np.std(all_individual_f1_scores)
        time_mean, time_std = np.mean(all_individual_times), np.std(all_individual_times)

    logging.info("\n===== OVERALL RESULTS (Averaged over all N_OUTER_SPLITS * N_INNER_SPLITS models) =====")
    logging.info(f"k value (n_neighbors): {k_value}")
    logging.info(f"Number of inner splits per outer fold: {N_INNER_SPLITS}")
    logging.info(f"Total individual models evaluated: {len(all_individual_accuracies)}")
    logging.info(f"Accuracy: {acc_mean:.7f} ± {acc_std:.7f}")
    logging.info(f"Precision: {prec_mean:.7f} ± {prec_std:.7f}")
    logging.info(f"Recall: {rec_mean:.7f} ± {rec_std:.7f}")
    logging.info(f"F1 Score: {f1_mean:.7f} ± {f1_std:.7f}")
    logging.info(f"Average execution time per inner model: {time_mean:.7f}s ± {time_std:.7f}s")

    plt.figure(figsize=(10, 6))
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    values = [acc_mean, prec_mean, rec_mean, f1_mean]
    errors = [acc_std, prec_std, rec_std, f1_std]
    bars = plt.bar(metrics, values, yerr=errors, capsize=10)
    plt.title(f'Overall Performance (Avg. of Inner Models, k={k_value}, Inner Splits={N_INNER_SPLITS})')
    plt.ylabel('Score'); plt.ylim(0, 1.1)
    for bar, val, err in zip(bars, values, errors):
        if not np.isnan(val) and not np.isnan(err):
            plt.text(bar.get_x() + bar.get_width()/2., val + err + 0.02 if val + err < 1.05 else 1.05,
                    f'{val:.7f}±{err:.7f}', ha='center', va='bottom', rotation=0)
    plt.tight_layout()
    plt.savefig(f'modified_knn_strategy3_k{k_value}_n{N_INNER_SPLITS}_overall_metrics.png')
    plt.close()

    if model_for_cm and X_cm is not None and y_cm is not None and len(y_cm) > 0:
        logging.info(f"analyze_results CM for {model_desc}: X_cm shape: {X_cm.shape}, y_cm shape: {y_cm.shape}")
        cm_predictions = model_for_cm.predict(X_cm)
        plt.figure(figsize=(8, 6))
        cm = confusion_matrix(y_cm, cm_predictions)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
        plt.title(f'Confusion Matrix ({model_desc}, k={k_value})')
        plt.xlabel('Predicted'); plt.ylabel('Actual')
        plt.savefig(f'modified_knn_strategy3_k{k_value}_n{N_INNER_SPLITS}_{model_desc.replace(" ","_")}_CM.png')
        plt.close()
    elif not model_for_cm: logging.warning(f"analyze_results: Model ({model_desc}) not found for CM.")
    else: logging.warning(f"analyze_results: X_cm or y_cm for CM ({model_desc}) is missing/empty.")


def main():
    global all_individual_accuracies, all_individual_precisions, all_individual_recalls
    global all_individual_f1_scores, all_individual_times, all_inner_model_details

    all_individual_accuracies.clear(); all_individual_precisions.clear(); all_individual_recalls.clear()
    all_individual_f1_scores.clear(); all_individual_times.clear(); all_inner_model_details.clear()

    overall_start_time = time.time()
    logging.info(f"Modified KNN testing begins (Strateji 3: Strategy 3: Typical Internal Model) (k={k_value}, N_INNER_SPLITS={N_INNER_SPLITS})...")

    fold_counter = 0
    for train_index, test_index in kf.split(X, y):
        fold_counter += 1
        logging.info(f"\n--- Outer Fold {fold_counter}/{N_OUTER_SPLITS} ---")
        X_train_full_orig, X_test_orig_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_full_orig, y_test_orig_fold = y.iloc[train_index], y.iloc[test_index]

        scaler = StandardScaler()
        X_train_full_scaled = scaler.fit_transform(X_train_full_orig)
        X_test_scaled_fold = scaler.transform(X_test_orig_fold)
        current_outer_fold_accuracies = []
        num_samples_in_full_train = len(X_train_full_scaled)
        actual_inner_splits = N_INNER_SPLITS
        subset_size = num_samples_in_full_train // actual_inner_splits

        if num_samples_in_full_train < N_INNER_SPLITS:
            logging.warning(f"Outer Fold {fold_counter}: Training data size ({num_samples_in_full_train}) is smaller than N_INNER_SPLITS ({N_INNER_SPLITS}). Reducing N_INNER_SPLITS to {num_samples_in_full_train}(each sample is a split).")
            actual_inner_splits = num_samples_in_full_train if num_samples_in_full_train > 0 else 1
            subset_size = 1

        if subset_size == 0 and actual_inner_splits > 0 and num_samples_in_full_train > 0 :
             logging.warning(f"Outer Fold {fold_counter}: Subset size is 0, num_samples_in_full_train={num_samples_in_full_train}, actual_inner_splits={actual_inner_splits}. Setting number of splits to 1.")
             actual_inner_splits = 1
             subset_size = num_samples_in_full_train
        elif num_samples_in_full_train == 0:
            logging.warning(f"Outer Fold {fold_counter}: Full training set is empty. Skipping this fold.")
            continue


        for i in range(actual_inner_splits):
            start_index = i * subset_size
            end_index = (i + 1) * subset_size if i < actual_inner_splits - 1 else num_samples_in_full_train
            if start_index >= end_index : continue


            X_train_subset = X_train_full_scaled[start_index:end_index]
            y_train_subset = y_train_full_orig.iloc[start_index:end_index]

            if len(X_train_subset) == 0:
                logging.warning(f"Outer Fold {fold_counter}, Inner Split {i+1}: Training subset is empty. Skipping.")
                continue

            n_neighbors_actual = min(k_value, len(X_train_subset))
            if n_neighbors_actual == 0 :
                logging.warning(f"Outer Fold {fold_counter}, Inner Split {i+1}: n_neighbors_actual became 0. Skipping.")
                continue

            unique_classes_in_subset = np.unique(y_train_subset)
            if len(unique_classes_in_subset) < 2 and len(y_train_subset) > 0 :
                 logging.warning(f"Outer Fold {fold_counter}, Inner Split {i+1}: Training subset has only one class ({unique_classes_in_subset}). KNN might make constant predictions.")
                 if len(y_train_subset) < n_neighbors_actual and len(y_train_subset) > 0:
                     n_neighbors_actual = len(y_train_subset)


            inner_model_start_time = time.time()
            knn = KNeighborsClassifier(n_neighbors=n_neighbors_actual)

            try:
                knn.fit(X_train_subset, y_train_subset)
                if not hasattr(knn, 'classes_') or len(knn.classes_) < 2 :
                    logging.warning(f"Outer Fold {fold_counter}, Inner Split {i+1}: KNN model learned only one class. Predictions might be uniform.")

                predictions = knn.predict(X_test_scaled_fold)
                acc = accuracy_score(y_test_orig_fold, predictions)
                prec = precision_score(y_test_orig_fold, predictions, average='weighted', zero_division=0)
                rec = recall_score(y_test_orig_fold, predictions, average='weighted', zero_division=0)
                f1 = f1_score(y_test_orig_fold, predictions, average='weighted', zero_division=0)
                inner_model_exec_time = time.time() - inner_model_start_time

                all_individual_accuracies.append(acc); all_individual_precisions.append(prec)
                all_individual_recalls.append(rec); all_individual_f1_scores.append(f1)
                all_individual_times.append(inner_model_exec_time)
                current_outer_fold_accuracies.append(acc)

                all_inner_model_details.append({
                    'model_object': knn, 'accuracy': acc, 'precision': prec, 'recall': rec, 'f1_score': f1,
                    'X_test_scaled_fold': X_test_scaled_fold, 'y_test_orig_fold': y_test_orig_fold,
                    'fold_num': fold_counter, 'inner_split_num': i + 1, 'train_subset_size': len(X_train_subset)
                })
            except Exception as e:
                logging.error(f"Outer Fold {fold_counter}, Inner Split {i+1}: Error during model training/prediction: {e}")

        if current_outer_fold_accuracies:
            mean_acc_fold, std_acc_fold = np.mean(current_outer_fold_accuracies), np.std(current_outer_fold_accuracies)
            logging.info(f"Outer Fold {fold_counter} Avg. Acc of Inner Models: {mean_acc_fold:.7f} (Std: {std_acc_fold:.7f})")
        else:
            logging.info(f"Outer Fold {fold_counter}: No valid inner model could be evaluated in this fold.")

    overall_end_time = time.time()
    logging.info(f"\nAll outer folds and inner splits completed. Total time: {overall_end_time - overall_start_time:.2f} seconds")

    model_for_analysis = None
    X_test_for_analysis = None
    y_test_for_analysis = None
    model_description_for_analysis = "N/A"

    if all_inner_model_details:
        if all_individual_accuracies:
            mean_overall_accuracy = np.mean(all_individual_accuracies)
            logging.info(f"Average accuracy of all inner models (for typical model selection): {mean_overall_accuracy:.7f}")

            valid_models_for_typical_selection = [m for m in all_inner_model_details if not np.isnan(m['accuracy'])]
            if valid_models_for_typical_selection:
                typical_inner_model_info = min(valid_models_for_typical_selection, key=lambda item: abs(item['accuracy'] - mean_overall_accuracy))
                model_for_analysis = typical_inner_model_info['model_object']
                X_test_for_analysis = typical_inner_model_info['X_test_scaled_fold']
                y_test_for_analysis = typical_inner_model_info['y_test_orig_fold']
                model_description_for_analysis = "Typical Inner Model"

                logging.info(f"\nIndividual INNER MODEL closest to the average (TYPICAL) selected (for Noise Test and CM):")
                logging.info(f"  Originating Outer Fold: {typical_inner_model_info['fold_num']}")
                logging.info(f"  Originating Inner Split: {typical_inner_model_info['inner_split_num']}")
                logging.info(f"  Training Subset Size: {typical_inner_model_info['train_subset_size']}")
                logging.info(f"  Accuracy of this Model (on its respective outer fold test set): {typical_inner_model_info['accuracy']:.7f}")
                logging.info(f"  This Model's F1 Score: {typical_inner_model_info['f1_score']:.7f}")
            else:
                logging.warning("A typical inner model close to the average could not be selected (no valid models).")
        else:
            logging.warning("Average accuracy could not be calculated, a typical model cannot be selected. The best model will be tried as a fallback.")
            if all_inner_model_details:
                 best_fallback_model_info = max(all_inner_model_details, key=lambda item: item['accuracy'])
                 model_for_analysis = best_fallback_model_info['model_object']
                 X_test_for_analysis = best_fallback_model_info['X_test_scaled_fold']
                 y_test_for_analysis = best_fallback_model_info['y_test_orig_fold']
                 model_description_for_analysis = "Best Inner Model (Fallback)"
                 logging.info(f"\nFALLBACK: Best Inner Model selected.")
                 logging.info(f"  Accuracy: {best_fallback_model_info['accuracy']:.7f}")


    else:
        logging.warning("No inner model details were recorded. A model could not be selected for Noise Test and CM.")

    analyze_results(model_for_cm=model_for_analysis,
                    X_cm=X_test_for_analysis,
                    y_cm=y_test_for_analysis,
                    model_desc=model_description_for_analysis)

    if model_for_analysis and X_test_for_analysis is not None and y_test_for_analysis is not None and len(y_test_for_analysis) > 0 :
        logging.info(f"\nRunning noise robustness test for the selected {model_description_for_analysis}...")
        noise_levels_config = [0.1, 0.2, 0.3, 0.4, 0.5]
        n_bootstrap_config = 100
        evaluate_robustness(
            model_for_analysis, X_test_for_analysis, y_test_for_analysis,
            noise_levels=noise_levels_config, n_bootstrap=n_bootstrap_config,
            model_desc=model_description_for_analysis
        )
    elif not model_for_analysis:
        logging.info(f"A suitable model ({model_description_for_analysis}) for the noise test could not be found.")
    else:
        logging.info(f"For the noise test, {model_description_for_analysis}'s test data is missing/empty.")

    logging.info("Process completed.")

if __name__ == "__main__":
    main()

## Fed3C (Bayesian Optimization)

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os
import argparse
from scipy.io import arff
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from skopt import gp_minimize
from skopt.space import Integer
from skopt.utils import use_named_args
import logging
from joblib import Parallel, delayed
from scipy import stats

args = type('', (), {})()
args.num_clients = 10
args.max_centroids = None
args.n_calls = 50
args.k = 1
args.experiment_name = "k1_rice_reference_experiment"

if args.experiment_name is None:
    exp_name = f"clients_{args.num_clients}_opt_{args.n_calls}_k_{args.k}"
    if args.max_centroids is not None:
        exp_name += f"_maxcentroid_{args.max_centroids}"
    args.experiment_name = exp_name

RESULTS_ROOT = "results"
RESULTS_DIR = os.path.join(RESULTS_ROOT, args.experiment_name)

os.makedirs(RESULTS_DIR, exist_ok=True)

log_file_path = os.path.join(RESULTS_DIR, 'rice_bayesian.log')

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[logging.FileHandler(log_file_path), logging.StreamHandler()])

logging.info(f"===== EXPERIMENT CONFIGURATION =====")
logging.info(f"Experiment Name: {args.experiment_name}")
logging.info(f"Number of Clients: {args.num_clients}")
logging.info(f"Optimization Calls: {args.n_calls}")
logging.info(f"KNN n_neighbors (k): {args.k}")
logging.info(f"Max Centroids: {'Half of training data' if args.max_centroids is None else args.max_centroids}")
logging.info(f"Results Directory: {RESULTS_DIR}")

data_path = 'rice/Rice_Cammeo_Osmancik.arff'
data, meta = arff.loadarff(data_path)
data_df = pd.DataFrame(data)

label_encoder = LabelEncoder()
data_df['Class'] = label_encoder.fit_transform(data_df['Class'].astype(str))

X = data_df.drop(columns=['Class'])
y = data_df['Class']

kf = KFold(n_splits=5, shuffle=True, random_state=42)
total_data = len(data_df)
train_data_per_fold = total_data * (4/5)

if args.max_centroids is not None:
    max_centroids = args.max_centroids
else:
    max_centroids = int(train_data_per_fold / 2)

logging.info(f"Calculated Max Centroids: {max_centroids}")

space = [Integer(10, max_centroids, name='total_centroids')]

best_accuracy = float('inf')
best_centroids = None
best_accuracies = []
best_precisions = []
best_recalls = []
best_f1s = []
best_times = []
best_client_times = []
best_comm_costs = []
best_y_test = None
best_predictions = None

all_centroids = []
all_accuracies = []
all_precisions = []
all_recalls = []
all_f1s = []
all_times = []
all_client_times = []
all_comm_costs = []

def log_details(split_data, centroids_per_split):
    for i, (data, centroids) in enumerate(zip(split_data, centroids_per_split)):
        logging.info(f"Client {i+1}: Data count = {len(data)}, Number of assigned centroids = {int(centroids)}")

def distribute_centroids(n_data, total_centroids):
    proportions = n_data / np.sum(n_data)
    floored_centroids = np.floor(proportions * total_centroids).astype(int)
    remainder = total_centroids - np.sum(floored_centroids)

    while remainder > 0:
        idx = np.argmax(proportions - floored_centroids / total_centroids)
        floored_centroids[idx] += 1
        remainder -= 1

    return floored_centroids

def calculate_communication_cost(centroids_per_split, feature_dim):
    bytes_per_centroid = feature_dim * 8
    total_bytes = sum(centroids_per_split) * bytes_per_centroid
    return total_bytes / 1024

def process_fold(train_index, test_index, total_centroids, X, y):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X.iloc[train_index])
    X_test_scaled = scaler.transform(X.iloc[test_index])
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    start_time = time.time()

    num_splits = args.num_clients
    split_train_data = np.array_split(X_train_scaled, num_splits)
    split_train_labels = np.array_split(y_train, num_splits)
    data_counts = np.array([len(data) for data in split_train_data])
    centroids_per_split = distribute_centroids(data_counts, total_centroids)

    center_labels = {}
    for split_data, split_labels, centroids_count in zip(split_train_data, split_train_labels, centroids_per_split):
        unique_labels, counts = np.unique(split_labels, return_counts=True)
        label_counts = dict(zip(unique_labels, counts / counts.sum()))
        centroids_per_class = {label: max(1, int(count * centroids_count))
                              for label, count in label_counts.items()}

        total_class_centroids = sum(centroids_per_class.values())
        if total_class_centroids > centroids_count:
            scale_factor = centroids_count / total_class_centroids
            centroids_per_class = {label: max(1, int(count * scale_factor))
                                  for label, count in centroids_per_class.items()}

        for label in unique_labels:
            class_data = split_data[split_labels == label]
            if centroids_per_class[label] > 0 and len(class_data) > 0:
                kmeans = KMeans(n_clusters=min(len(class_data), centroids_per_class[label]), random_state=42)
                kmeans.fit(class_data)
                if label in center_labels:
                    center_labels[label].extend(kmeans.cluster_centers_)
                else:
                    center_labels[label] = list(kmeans.cluster_centers_)

    final_centers = [center for sublist in center_labels.values() for center in sublist]
    final_labels = [label for label, centers in center_labels.items() for center in centers]

    knn = KNeighborsClassifier(n_neighbors=args.k)
    knn.fit(final_centers, final_labels)
    predictions = knn.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    f1 = f1_score(y_test, predictions, average='weighted')

    end_time = time.time()
    exec_time = end_time - start_time

    feature_dim = X.shape[1]
    comm_cost = calculate_communication_cost(centroids_per_split, feature_dim)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'time': exec_time,
        'comm_cost': comm_cost,
        'y_test': y_test,
        'predictions': predictions
    }

@use_named_args(space)
def objective_sequential(total_centroids):
    logging.info(f"Total number of centroids tried = {total_centroids}")

    fold_accuracies = []
    fold_precisions = []
    fold_recalls = []
    fold_f1s = []
    fold_times = []
    fold_client_times = []
    fold_comm_costs = []

    for train_index, test_index in kf.split(X):
        fold_start_time = time.time()

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X.iloc[train_index])
        X_test_scaled = scaler.transform(X.iloc[test_index])
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        num_splits = args.num_clients
        split_train_data = np.array_split(X_train_scaled, num_splits)
        split_train_labels = np.array_split(y_train, num_splits)
        data_counts = np.array([len(data) for data in split_train_data])
        centroids_per_split = distribute_centroids(data_counts, total_centroids)

        center_labels = {}
        client_times = []

        for split_data, split_labels, centroids_count in zip(split_train_data, split_train_labels, centroids_per_split):
            client_start_time = time.time()

            unique_labels, counts = np.unique(split_labels, return_counts=True)
            label_counts = dict(zip(unique_labels, counts / counts.sum()))
            centroids_per_class = {label: max(1, int(count * centroids_count))
                                  for label, count in label_counts.items()}

            total_class_centroids = sum(centroids_per_class.values())
            if total_class_centroids > centroids_count:
                scale_factor = centroids_count / total_class_centroids
                centroids_per_class = {label: max(1, int(count * scale_factor))
                                      for label, count in centroids_per_class.items()}

            for label in unique_labels:
                class_data = split_data[split_labels == label]
                if centroids_per_class[label] > 0 and len(class_data) > 0:
                    kmeans = KMeans(n_clusters=min(len(class_data), centroids_per_class[label]), random_state=42)
                    kmeans.fit(class_data)
                    if label in center_labels:
                        center_labels[label].extend(kmeans.cluster_centers_)
                    else:
                        center_labels[label] = list(kmeans.cluster_centers_)

            client_end_time = time.time()
            client_times.append(client_end_time - client_start_time)

        final_centers = [center for sublist in center_labels.values() for center in sublist]
        final_labels = [label for label, centers in center_labels.items() for center in centers]

        knn = KNeighborsClassifier(n_neighbors=args.k)
        knn.fit(final_centers, final_labels)
        predictions = knn.predict(X_test_scaled)

        accuracy = accuracy_score(y_test, predictions)
        precision = precision_score(y_test, predictions, average='weighted')
        recall = recall_score(y_test, predictions, average='weighted')
        f1 = f1_score(y_test, predictions, average='weighted')

        fold_end_time = time.time()
        fold_exec_time = fold_end_time - fold_start_time

        feature_dim = X.shape[1]
        comm_cost = calculate_communication_cost(centroids_per_split, feature_dim)

        fold_accuracies.append(accuracy)
        fold_precisions.append(precision)
        fold_recalls.append(recall)
        fold_f1s.append(f1)
        fold_times.append(fold_exec_time)
        fold_client_times.append(client_times)
        fold_comm_costs.append(comm_cost)

    mean_accuracy = np.mean(fold_accuracies)
    mean_precision = np.mean(fold_precisions)
    mean_recall = np.mean(fold_recalls)
    mean_f1 = np.mean(fold_f1s)
    mean_time = np.mean(fold_times)
    mean_client_time = np.mean([np.mean(client_times) for client_times in fold_client_times])
    mean_comm_cost = np.mean(fold_comm_costs)

    global best_accuracy, best_centroids, best_accuracies, best_precisions, best_recalls, best_f1s, best_times, best_client_times, best_comm_costs, best_y_test, best_predictions

    all_centroids.append(total_centroids)
    all_accuracies.append(mean_accuracy)
    all_precisions.append(mean_precision)
    all_recalls.append(mean_recall)
    all_f1s.append(mean_f1)
    all_times.append(mean_time)
    all_client_times.append(mean_client_time)
    all_comm_costs.append(mean_comm_cost)

    if -mean_accuracy < best_accuracy:
        best_accuracy = -mean_accuracy
        best_centroids = total_centroids
        best_accuracies = fold_accuracies
        best_precisions = fold_precisions
        best_recalls = fold_recalls
        best_f1s = fold_f1s
        best_times = fold_times
        best_client_times = [np.mean(client_times) for client_times in fold_client_times]
        best_comm_costs = fold_comm_costs

        # Test
        if len(fold_accuracies) > 0:
            best_fold_idx = np.argmax(fold_accuracies)
            best_fold_indices = list(kf.split(X))[best_fold_idx]
            test_indices = best_fold_indices[1]
            best_y_test = y.iloc[test_indices]

            best_X_test = X.iloc[test_indices]
            best_X_test_scaled = scaler.transform(best_X_test)
            best_predictions = knn.predict(best_X_test_scaled)

    logging.info(f"Centroids: {total_centroids}, Acc: {mean_accuracy:.7f}, Prec: {mean_precision:.7f}, " +
                 f"Rec: {mean_recall:.7f}, F1: {mean_f1:.7f}, Time (fold): {mean_time:.2f}s, " +
                 f"Time (client): {mean_client_time:.7f}s, Comm: {mean_comm_cost:.2f}KB")

    return -mean_accuracy

def evaluate_robustness(best_model, X_test, y_test, noise_levels=[0.1, 0.2, 0.3, 0.4, 0.5], n_bootstrap=100):
    results = []

    n_samples = len(y_test)

    original_pred = best_model.predict(X_test)
    original_acc = accuracy_score(y_test, original_pred)
    original_f1 = f1_score(y_test, original_pred, average='weighted')
    results.append(('No Noise', original_acc, original_f1))

    bootstrap_original_acc = []
    bootstrap_original_f1 = []

    for _ in range(n_bootstrap):
        indices = np.random.choice(n_samples, n_samples, replace=True)
        X_boot = X_test[indices]
        y_boot = y_test.iloc[indices] if hasattr(y_test, 'iloc') else y_test[indices]

        boot_pred = best_model.predict(X_boot)
        bootstrap_original_acc.append(accuracy_score(y_boot, boot_pred))
        bootstrap_original_f1.append(f1_score(y_boot, boot_pred, average='weighted'))

    p_values_acc = []
    p_values_f1 = []
    bootstrap_acc_all = []
    bootstrap_f1_all = []

    for noise in noise_levels:
        bootstrap_acc = []
        bootstrap_f1 = []

        for _ in range(n_bootstrap):
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_boot = X_test[indices].copy()
            y_boot = y_test.iloc[indices] if hasattr(y_test, 'iloc') else y_test[indices]

            noise_matrix = np.random.normal(0, noise, X_boot.shape)
            X_boot += noise_matrix

            noisy_pred = best_model.predict(X_boot)
            noisy_acc = accuracy_score(y_boot, noisy_pred)
            noisy_f1 = f1_score(y_boot, noisy_pred, average='weighted')

            bootstrap_acc.append(noisy_acc)
            bootstrap_f1.append(noisy_f1)

        mean_acc = np.mean(bootstrap_acc)
        mean_f1 = np.mean(bootstrap_f1)

        t_stat_acc, p_val_acc = stats.ttest_rel(bootstrap_original_acc, bootstrap_acc)
        t_stat_f1, p_val_f1 = stats.ttest_rel(bootstrap_original_f1, bootstrap_f1)

        results.append((f'Noise {noise:.2f}', mean_acc, mean_f1))
        p_values_acc.append(p_val_acc)
        p_values_f1.append(p_val_f1)
        bootstrap_acc_all.append(bootstrap_acc)
        bootstrap_f1_all.append(bootstrap_f1)

    plt.figure(figsize=(12, 7))
    noise_labels = [r[0] for r in results]
    acc_values = [r[1] for r in results]
    f1_values = [r[2] for r in results]

    x = np.arange(len(noise_labels))
    width = 0.35

    plt.subplot(1, 2, 1)
    bars_acc = plt.bar(x - width/2, acc_values, width, label='Accuracy')
    bars_f1 = plt.bar(x + width/2, f1_values, width, label='F1 Score')
    plt.xlabel('Noise Level')
    plt.ylabel('Score')
    plt.title('Model Robustness to Noise')
    plt.xticks(x, noise_labels, rotation=45)
    plt.legend()

    for i in range(len(p_values_acc)):
        if i > 0:
            if p_values_acc[i-1] < 0.001:
                plt.text(i - width/2, acc_values[i] - 0.02, '***', ha='center')
            elif p_values_acc[i-1] < 0.01:
                plt.text(i - width/2, acc_values[i] - 0.02, '**', ha='center')
            elif p_values_acc[i-1] < 0.05:
                plt.text(i - width/2, acc_values[i] - 0.02, '*', ha='center')

            if p_values_f1[i-1] < 0.001:
                plt.text(i + width/2, f1_values[i] - 0.02, '***', ha='center')
            elif p_values_f1[i-1] < 0.01:
                plt.text(i + width/2, f1_values[i] - 0.02, '**', ha='center')
            elif p_values_f1[i-1] < 0.05:
                plt.text(i + width/2, f1_values[i] - 0.02, '*', ha='center')

    plt.subplot(1, 2, 2)
    bars_acc = plt.bar(x - width/2, acc_values, width, label='Accuracy')
    bars_f1 = plt.bar(x + width/2, f1_values, width, label='F1 Score')
    plt.xlabel('Noise Level')
    plt.ylabel('Score')
    plt.title('Zoomed View (0.85-1.0 range) with p-values')
    plt.xticks(x, noise_labels, rotation=45)
    plt.ylim(0.85, 1.0)

    for i in range(len(p_values_acc)):
        if i > 0:
            if p_values_acc[i-1] < 0.001:
                plt.text(i - width/2, acc_values[i] + 0.005, '***', ha='center', fontsize=10)
            elif p_values_acc[i-1] < 0.01:
                plt.text(i - width/2, acc_values[i] + 0.005, '**', ha='center', fontsize=10)
            elif p_values_acc[i-1] < 0.05:
                plt.text(i - width/2, acc_values[i] + 0.005, '*', ha='center', fontsize=10)

            if p_values_f1[i-1] < 0.001:
                plt.text(i + width/2, f1_values[i] + 0.005, '***', ha='center', fontsize=10)
            elif p_values_f1[i-1] < 0.01:
                plt.text(i + width/2, f1_values[i] + 0.005, '**', ha='center', fontsize=10)
            elif p_values_f1[i-1] < 0.05:
                plt.text(i + width/2, f1_values[i] + 0.005, '*', ha='center', fontsize=10)

    for i, (acc, f1) in enumerate(zip(acc_values, f1_values)):
        plt.text(i - width/2, acc - 0.01, f'{acc:.7f}', ha='center', va='bottom', fontsize=8, rotation=90)
        plt.text(i + width/2, f1 - 0.01, f'{f1:.7f}', ha='center', va='bottom', fontsize=8, rotation=90)

    plt.figtext(0.5, 0.01, "* p<0.05, ** p<0.01, *** p<0.001", ha="center", fontsize=10)

    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'rice_noise_robustness.png'))
    plt.close()

    plt.figure(figsize=(14, 8))

    plt.subplot(2, 1, 1)
    boxplot_data_acc = [bootstrap_original_acc] + bootstrap_acc_all
    plt.boxplot(boxplot_data_acc, labels=noise_labels, showfliers=False)
    plt.title('Distribution of Accuracy Scores Across Noise Levels')
    plt.ylabel('Accuracy')
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.subplot(2, 1, 2)
    boxplot_data_f1 = [bootstrap_original_f1] + bootstrap_f1_all
    plt.boxplot(boxplot_data_f1, labels=noise_labels, showfliers=False)
    plt.title('Distribution of F1 Scores Across Noise Levels')
    plt.ylabel('F1 Score')
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'rice_noise_distribution.png'))
    plt.close()

    logging.info("\n===== NOISE ROBUSTNESS RESULTS WITH P-VALUES =====")
    logging.info(f"{'Noise Level':<12} {'Accuracy':<15} {'F1 Score':<15} {'p-val (Acc)':<12} {'p-val (F1)':<12}")

    logging.info(f"{noise_labels[0]:<12} {acc_values[0]:.7f}      {f1_values[0]:.7f}      -            -")

    for i in range(1, len(noise_labels)):
        logging.info(f"{noise_labels[i]:<12} {acc_values[i]:.7f}      {f1_values[i]:.7f}      {p_values_acc[i-1]:.2e}     {p_values_f1[i-1]:.2e}")

    return results, p_values_acc, p_values_f1

def analyze_results():
    acc_mean, acc_std = np.mean(best_accuracies), np.std(best_accuracies)
    prec_mean, prec_std = np.mean(best_precisions), np.std(best_precisions)
    rec_mean, rec_std = np.mean(best_recalls), np.std(best_recalls)
    f1_mean, f1_std = np.mean(best_f1s), np.std(best_f1s)
    time_mean, time_std = np.mean(best_times), np.std(best_times)
    client_time_mean, client_time_std = np.mean(best_client_times), np.std(best_client_times)
    comm_cost_mean, comm_cost_std = np.mean(best_comm_costs), np.std(best_comm_costs)

    logging.info("\n===== BEST CONFIGURATION RESULTS (CLASS-BALANCED CENTROID ALLOCATION) =====")
    logging.info(f"Best number of centroids: {best_centroids}")
    logging.info(f"Number of clients: {args.num_clients}")
    logging.info(f"KNN n_neighbors (k): {args.k}")
    logging.info(f"Centroid allocation strategy: Class-balanced (Allocation to each class proportional to its data size)")
    logging.info(f"Accuracy: {acc_mean:.7f} ± {acc_std:.7f}")
    logging.info(f"Precision: {prec_mean:.7f} ± {prec_std:.7f}")
    logging.info(f"Recall: {rec_mean:.7f} ± {rec_std:.7f}")
    logging.info(f"F1 Score: {f1_mean:.7f} ± {f1_std:.7f}")
    logging.info(f"Average time per fold: {time_mean:.7f}s ± {time_std:.7f}s")
    logging.info(f"Average time per client: {client_time_mean:.7f}s ± {client_time_std:.7f}s")
    logging.info(f"Communication cost: {comm_cost_mean:.7f}KB ± {comm_cost_std:.7f}KB")

    if best_y_test is not None:
        unique_labels, counts = np.unique(best_y_test, return_counts=True)
        class_ratios = dict(zip(unique_labels, counts / counts.sum()))

        for label, ratio in class_ratios.items():
            class_name = label_encoder.inverse_transform([label])[0] if hasattr(label_encoder, 'inverse_transform') else f"Class {label}"
            logging.info(f"Class ratio ({class_name}): {ratio:.7f}")

    plt.figure(figsize=(10, 6))
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    values = [acc_mean, prec_mean, rec_mean, f1_mean]
    errors = [acc_std, prec_std, rec_std, f1_std]

    bars = plt.bar(metrics, values, yerr=errors, capsize=10)
    plt.title(f'Performance Metrics with Error Bars (Centroids: {best_centroids}, Clients: {args.num_clients})')
    plt.ylabel('Score')
    plt.ylim(0, 1.1)

    for bar, val, err in zip(bars, values, errors):
        plt.text(bar.get_x() + bar.get_width()/2., val + err + 0.02,
                f'{val:.7f}±{err:.7f}', ha='center', va='bottom', rotation=0)

    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'rice_performance_metrics.png'))
    plt.close()

    if best_y_test is not None and best_predictions is not None:
        plt.figure(figsize=(8, 6))
        cm = confusion_matrix(best_y_test, best_predictions)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f'Confusion Matrix (Centroids: {best_centroids}, Clients: {args.num_clients})')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.savefig(os.path.join(RESULTS_DIR, 'rice_confusion_matrix.png'))
        plt.close()

    plt.figure(figsize=(14, 8))

    plt.subplot(2, 2, 1)
    plt.plot(range(len(all_centroids)), all_accuracies, 'b-', label='Accuracy')
    plt.plot(range(len(all_centroids)), all_f1s, 'g-', label='F1 Score')
    plt.axvline(x=all_centroids.index(best_centroids), color='r', linestyle='--', label=f'Best Centroids={best_centroids}')
    plt.title('Accuracy and F1 Score vs. Iteration')
    plt.xlabel('Iteration')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.subplot(2, 2, 2)
    plt.plot(range(len(all_centroids)), all_precisions, 'm-', label='Precision')
    plt.plot(range(len(all_centroids)), all_recalls, 'c-', label='Recall')
    plt.axvline(x=all_centroids.index(best_centroids), color='r', linestyle='--', label=f'Best Centroids={best_centroids}')
    plt.title('Precision and Recall vs. Iteration')
    plt.xlabel('Iteration')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.subplot(2, 2, 3)
    plt.plot(range(len(all_centroids)), all_times, 'k-', label='Fold Time')
    plt.plot(range(len(all_centroids)), all_client_times, 'y-', label='Client Time')
    plt.axvline(x=all_centroids.index(best_centroids), color='r', linestyle='--', label=f'Best Centroids={best_centroids}')
    plt.title('Execution Time vs. Iteration')
    plt.xlabel('Iteration')
    plt.ylabel('Time (s)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.subplot(2, 2, 4)
    plt.plot(range(len(all_centroids)), all_comm_costs, 'r-', label='Comm Cost')
    plt.axvline(x=all_centroids.index(best_centroids), color='b', linestyle='--', label=f'Best Centroids={best_centroids}')
    plt.title('Communication Cost vs. Iteration')
    plt.xlabel('Iteration')
    plt.ylabel('Communication Cost (KB)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'rice_optimization_progress.png'))
    plt.close()

    return {
        'acc_mean': acc_mean, 'acc_std': acc_std,
        'prec_mean': prec_mean, 'prec_std': prec_std,
        'rec_mean': rec_mean, 'rec_std': rec_std,
        'f1_mean': f1_mean, 'f1_std': f1_std,
        'time_mean': time_mean, 'time_std': time_std,
        'client_time_mean': client_time_mean, 'client_time_std': client_time_std,
        'comm_cost_mean': comm_cost_mean, 'comm_cost_std': comm_cost_std
    }

def main():
    start_time = time.time()

    logging.info(f"Bayesian optimization starting (without parallelization, with class-balanced centroid allocation)...")
    logging.info(f"Parameter configuration: Number of Clients={args.num_clients}, Max Centroid={max_centroids}, Optimization Iterations={args.n_calls}")

    res_gp = gp_minimize(objective_sequential, space, n_calls=args.n_calls, random_state=0)

    total_time = time.time() - start_time
    logging.info(f"Optimization completed. Total time: {total_time:.7f} seconds")

    metrics = analyze_results()

    logging.info("Running noise robustness test...")

    best_fold_idx = np.argmax(best_accuracies)
    train_indices = list(kf.split(X))[best_fold_idx][0]
    test_indices = list(kf.split(X))[best_fold_idx][1]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X.iloc[train_indices])
    X_test_scaled = scaler.transform(X.iloc[test_indices])
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]

    client_start_time = time.time()

    num_splits = args.num_clients
    split_train_data = np.array_split(X_train_scaled, num_splits)
    split_train_labels = np.array_split(y_train, num_splits)
    data_counts = np.array([len(data) for data in split_train_data])
    centroids_per_split = distribute_centroids(data_counts, best_centroids)

    client_times = []

    center_labels = {}
    for split_data, split_labels, centroids_count in zip(split_train_data, split_train_labels, centroids_per_split):
        client_iter_start = time.time()

        unique_labels, counts = np.unique(split_labels, return_counts=True)
        label_counts = dict(zip(unique_labels, counts / counts.sum()))
        centroids_per_class = {label: max(1, int(count * centroids_count))
                              for label, count in label_counts.items()}

        total_class_centroids = sum(centroids_per_class.values())
        if total_class_centroids > centroids_count:
            scale_factor = centroids_count / total_class_centroids
            centroids_per_class = {label: max(1, int(count * scale_factor))
                                   for label, count in centroids_per_class.items()}

        logging.info(f"Centroid distribution: {centroids_per_class}")

        for label in unique_labels:
            class_data = split_data[split_labels == label]
            if centroids_per_class[label] > 0 and len(class_data) > 0:
                kmeans = KMeans(n_clusters=min(len(class_data), centroids_per_class[label]), random_state=42)
                kmeans.fit(class_data)
                if label in center_labels:
                    center_labels[label].extend(kmeans.cluster_centers_)
                else:
                    center_labels[label] = list(kmeans.cluster_centers_)

        client_iter_end = time.time()
        client_times.append(client_iter_end - client_iter_start)

    client_end_time = time.time()
    client_total_time = client_end_time - client_start_time
    client_avg_time = np.mean(client_times)

    feature_dim = X.shape[1]
    comm_cost = calculate_communication_cost(centroids_per_split, feature_dim)

    logging.info(f"Client training times: Total={client_total_time:.7f}s, Average={client_avg_time:.7f}s")
    logging.info(f"Per client communication cost: {comm_cost/num_splits:.7f}KB")

    final_centers = [center for sublist in center_labels.values() for center in sublist]
    final_labels = [label for label, centers in center_labels.items() for center in centers]

    best_model = KNeighborsClassifier(n_neighbors=args.k)
    best_model.fit(final_centers, final_labels)

    noise_levels = [0.1, 0.2, 0.3, 0.4, 0.5]
    n_bootstrap = 100

    noise_results, p_values_acc, p_values_f1 = evaluate_robustness(
        best_model, X_test_scaled, y_test, noise_levels, n_bootstrap)

    summary_file = os.path.join(RESULTS_DIR, 'experiment_summary.txt')
    with open(summary_file, 'w') as f:
        f.write(f"===== EXPERIMENT SUMMARY =====\n")
        f.write(f"Experiment Name: {args.experiment_name}\n")
        f.write(f"Number of Clients: {args.num_clients}\n")
        f.write(f"KNN n_neighbors (k): {args.k}\n")
        f.write(f"Max Centroids: {max_centroids}\n")
        f.write(f"Optimization Iterations: {args.n_calls}\n")
        f.write(f"Total Execution Time: {total_time:.2f} seconds\n\n")

        f.write(f"===== BEST RESULTS =====\n")
        f.write(f"Best Centroids: {best_centroids}\n")
        f.write(f"Accuracy: {metrics['acc_mean']:.7f} ± {metrics['acc_std']:.7f}\n")
        f.write(f"Precision: {metrics['prec_mean']:.7f} ± {metrics['prec_std']:.7f}\n")
        f.write(f"Recall: {metrics['rec_mean']:.7f} ± {metrics['rec_std']:.7f}\n")
        f.write(f"F1 Score: {metrics['f1_mean']:.7f} ± {metrics['f1_std']:.7f}\n")
        f.write(f"Client Time: {metrics['client_time_mean']:.7f}s ± {metrics['client_time_std']:.7f}s\n")
        f.write(f"Communication Cost: {metrics['comm_cost_mean']:.7f}KB ± {metrics['comm_cost_std']:.7f}KB\n")

    logging.info(f"Process completed. Results were saved to the {RESULTS_DIR} folder.")
    logging.info(f"Experiment summary: {summary_file}")

if __name__ == "__main__":
    main()

## Fed3C (ABC)

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os
import argparse
import random  # ABC algoritması için eklendi
from scipy.io import arff
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
import logging
from joblib import Parallel, delayed
from scipy import stats

args = type('', (), {})()
args.num_clients = 10
args.max_centroids = None
args.n_calls = 50
args.n_bees = 10
args.k = 1
args.experiment_name = "k1_rice_abc_experiment"

if args.experiment_name is None:
    exp_name = f"clients_{args.num_clients}_abc_{args.n_calls}_bees_{args.n_bees}_k_{args.k}"
    if args.max_centroids is not None:
        exp_name += f"_maxcentroid_{args.max_centroids}"
    args.experiment_name = exp_name

RESULTS_ROOT = "results"
RESULTS_DIR = os.path.join(RESULTS_ROOT, args.experiment_name)

os.makedirs(RESULTS_DIR, exist_ok=True)

log_file_path = os.path.join(RESULTS_DIR, 'rice_abc.log')

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[logging.FileHandler(log_file_path), logging.StreamHandler()])

logging.info(f"===== EXPERIMENT CONFIGURATION =====")
logging.info(f"Experiment Name: {args.experiment_name}")
logging.info(f"Number of Clients: {args.num_clients}")
logging.info(f"Optimization Algorithm: Artificial Bee Colony (ABC)")
logging.info(f"Number of Bees: {args.n_bees}")
logging.info(f"Optimization Max Iterations: {args.n_calls}")
logging.info(f"KNN n_neighbors (k): {args.k}")
logging.info(f"Max Centroids: {'Half of training data' if args.max_centroids is None else args.max_centroids}")
logging.info(f"Results Directory: {RESULTS_DIR}")

data_path = 'rice/Rice_Cammeo_Osmancik.arff'
data, meta = arff.loadarff(data_path)
data_df = pd.DataFrame(data)

label_encoder = LabelEncoder()
data_df['Class'] = label_encoder.fit_transform(data_df['Class'].astype(str))

X = data_df.drop(columns=['Class'])
y = data_df['Class']

kf = KFold(n_splits=5, shuffle=True, random_state=42)
total_data = len(data_df)
train_data_per_fold = total_data * (4/5)

if args.max_centroids is not None:
    max_centroids = args.max_centroids
else:
    max_centroids = int(train_data_per_fold / 2)

logging.info(f"Calculated Max Centroids: {max_centroids}")

best_accuracy = float('-inf')
best_centroids = None
best_accuracies = []
best_precisions = []
best_recalls = []
best_f1s = []
best_times = []
best_client_times = []
best_comm_costs = []
best_y_test = None
best_predictions = None

all_centroids = []
all_accuracies = []
all_precisions = []
all_recalls = []
all_f1s = []
all_times = []
all_client_times = []
all_comm_costs = []

def log_details(split_data, centroids_per_split):
    for i, (data, centroids) in enumerate(zip(split_data, centroids_per_split)):
        logging.info(f"Client {i+1}: Data count = {len(data)}, Number of assigned centroids = {int(centroids)}")

def distribute_centroids(n_data, total_centroids):
    proportions = n_data / np.sum(n_data)
    floored_centroids = np.floor(proportions * total_centroids).astype(int)
    remainder = total_centroids - np.sum(floored_centroids)

    while remainder > 0:
        idx = np.argmax(proportions - floored_centroids / total_centroids)
        floored_centroids[idx] += 1
        remainder -= 1

    return floored_centroids

def calculate_communication_cost(centroids_per_split, feature_dim):
    bytes_per_centroid = feature_dim * 8
    total_bytes = sum(centroids_per_split) * bytes_per_centroid
    return total_bytes / 1024

def process_fold(train_index, test_index, total_centroids, X, y):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X.iloc[train_index])
    X_test_scaled = scaler.transform(X.iloc[test_index])
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    start_time = time.time()

    num_splits = args.num_clients
    split_train_data = np.array_split(X_train_scaled, num_splits)
    split_train_labels = np.array_split(y_train, num_splits)
    data_counts = np.array([len(data) for data in split_train_data])
    centroids_per_split = distribute_centroids(data_counts, total_centroids)

    center_labels = {}
    for split_data, split_labels, centroids_count in zip(split_train_data, split_train_labels, centroids_per_split):
        unique_labels, counts = np.unique(split_labels, return_counts=True)
        label_counts = dict(zip(unique_labels, counts / counts.sum()))
        centroids_per_class = {label: max(1, int(count * centroids_count))
                              for label, count in label_counts.items()}

        total_class_centroids = sum(centroids_per_class.values())
        if total_class_centroids > centroids_count:
            scale_factor = centroids_count / total_class_centroids
            centroids_per_class = {label: max(1, int(count * scale_factor))
                                  for label, count in centroids_per_class.items()}

        for label in unique_labels:
            class_data = split_data[split_labels == label]
            if centroids_per_class[label] > 0 and len(class_data) > 0:
                kmeans = KMeans(n_clusters=min(len(class_data), centroids_per_class[label]), random_state=42)
                kmeans.fit(class_data)
                if label in center_labels:
                    center_labels[label].extend(kmeans.cluster_centers_)
                else:
                    center_labels[label] = list(kmeans.cluster_centers_)

    final_centers = [center for sublist in center_labels.values() for center in sublist]
    final_labels = [label for label, centers in center_labels.items() for center in centers]

    knn = KNeighborsClassifier(n_neighbors=args.k)
    knn.fit(final_centers, final_labels)
    predictions = knn.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    f1 = f1_score(y_test, predictions, average='weighted')

    end_time = time.time()
    exec_time = end_time - start_time

    feature_dim = X.shape[1]
    comm_cost = calculate_communication_cost(centroids_per_split, feature_dim)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'time': exec_time,
        'comm_cost': comm_cost,
        'y_test': y_test,
        'predictions': predictions
    }

def objective_sequential(total_centroids):
    logging.info(f"Total number of centroids tried = {total_centroids}")

    fold_accuracies = []
    fold_precisions = []
    fold_recalls = []
    fold_f1s = []
    fold_times = []
    fold_client_times = []
    fold_comm_costs = []

    for train_index, test_index in kf.split(X):
        fold_start_time = time.time()

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X.iloc[train_index])
        X_test_scaled = scaler.transform(X.iloc[test_index])
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        num_splits = args.num_clients
        split_train_data = np.array_split(X_train_scaled, num_splits)
        split_train_labels = np.array_split(y_train, num_splits)
        data_counts = np.array([len(data) for data in split_train_data])
        centroids_per_split = distribute_centroids(data_counts, total_centroids)

        center_labels = {}
        client_times = []

        for split_data, split_labels, centroids_count in zip(split_train_data, split_train_labels, centroids_per_split):
            client_start_time = time.time()

            unique_labels, counts = np.unique(split_labels, return_counts=True)
            label_counts = dict(zip(unique_labels, counts / counts.sum()))
            centroids_per_class = {label: max(1, int(count * centroids_count))
                                  for label, count in label_counts.items()}

            total_class_centroids = sum(centroids_per_class.values())
            if total_class_centroids > centroids_count:
                scale_factor = centroids_count / total_class_centroids
                centroids_per_class = {label: max(1, int(count * scale_factor))
                                      for label, count in centroids_per_class.items()}

            for label in unique_labels:
                class_data = split_data[split_labels == label]
                if centroids_per_class[label] > 0 and len(class_data) > 0:
                    kmeans = KMeans(n_clusters=min(len(class_data), centroids_per_class[label]), random_state=42)
                    kmeans.fit(class_data)
                    if label in center_labels:
                        center_labels[label].extend(kmeans.cluster_centers_)
                    else:
                        center_labels[label] = list(kmeans.cluster_centers_)

            client_end_time = time.time()
            client_times.append(client_end_time - client_start_time)

        final_centers = [center for sublist in center_labels.values() for center in sublist]
        final_labels = [label for label, centers in center_labels.items() for center in centers]

        knn = KNeighborsClassifier(n_neighbors=args.k)
        knn.fit(final_centers, final_labels)
        predictions = knn.predict(X_test_scaled)

        accuracy = accuracy_score(y_test, predictions)
        precision = precision_score(y_test, predictions, average='weighted')
        recall = recall_score(y_test, predictions, average='weighted')
        f1 = f1_score(y_test, predictions, average='weighted')

        fold_end_time = time.time()
        fold_exec_time = fold_end_time - fold_start_time

        feature_dim = X.shape[1]
        comm_cost = calculate_communication_cost(centroids_per_split, feature_dim)

        fold_accuracies.append(accuracy)
        fold_precisions.append(precision)
        fold_recalls.append(recall)
        fold_f1s.append(f1)
        fold_times.append(fold_exec_time)
        fold_client_times.append(client_times)
        fold_comm_costs.append(comm_cost)

    mean_accuracy = np.mean(fold_accuracies)
    mean_precision = np.mean(fold_precisions)
    mean_recall = np.mean(fold_recalls)
    mean_f1 = np.mean(fold_f1s)
    mean_time = np.mean(fold_times)
    mean_client_time = np.mean([np.mean(client_times) for client_times in fold_client_times])
    mean_comm_cost = np.mean(fold_comm_costs)

    global best_accuracy, best_centroids, best_accuracies, best_precisions, best_recalls, best_f1s, best_times, best_client_times, best_comm_costs, best_y_test, best_predictions

    all_centroids.append(total_centroids)
    all_accuracies.append(mean_accuracy)
    all_precisions.append(mean_precision)
    all_recalls.append(mean_recall)
    all_f1s.append(mean_f1)
    all_times.append(mean_time)
    all_client_times.append(mean_client_time)
    all_comm_costs.append(mean_comm_cost)

    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_centroids = total_centroids
        best_accuracies = fold_accuracies
        best_precisions = fold_precisions
        best_recalls = fold_recalls
        best_f1s = fold_f1s
        best_times = fold_times
        best_client_times = [np.mean(client_times) for client_times in fold_client_times]
        best_comm_costs = fold_comm_costs

        if len(fold_accuracies) > 0:
            best_fold_idx = np.argmax(fold_accuracies)
            best_fold_indices = list(kf.split(X))[best_fold_idx]
            test_indices = best_fold_indices[1]
            best_y_test = y.iloc[test_indices]

            best_X_test = X.iloc[test_indices]
            best_X_test_scaled = scaler.transform(best_X_test)
            best_predictions = knn.predict(best_X_test_scaled)

    logging.info(f"Centroids: {total_centroids}, Acc: {mean_accuracy:.7f}, Prec: {mean_precision:.7f}, " +
                 f"Rec: {mean_recall:.7f}, F1: {mean_f1:.7f}, Time (fold): {mean_time:.2f}s, " +
                 f"Time (client): {mean_client_time:.7f}s, Comm: {mean_comm_cost:.2f}KB")

    return mean_accuracy, fold_accuracies

def abc_algorithm(objective, dimension_bounds, n_bees=30, max_iter=100):
    lower_bound, upper_bound = dimension_bounds

    population = [random.randint(lower_bound, upper_bound) for _ in range(n_bees)]
    fitness_values = []
    all_fold_accuracies = []

    for centroid in population:
        fitness, fold_accuracies = objective(centroid)
        fitness_values.append(fitness)
        all_fold_accuracies.append(fold_accuracies)

    for iter in range(max_iter):
        logging.info(f"ABC Iteration {iter+1}/{max_iter}")

        for i in range(n_bees):
            candidate = population[i] + random.randint(-5, 5)
            candidate = max(lower_bound, min(candidate, upper_bound))

            candidate_fitness, candidate_fold_accuracies = objective(candidate)

            if candidate_fitness > fitness_values[i]:
                population[i] = candidate
                fitness_values[i] = candidate_fitness
                all_fold_accuracies[i] = candidate_fold_accuracies

    best_index = fitness_values.index(max(fitness_values))
    best_centroids = population[best_index]
    best_accuracy = fitness_values[best_index]
    best_fold_accuracies = all_fold_accuracies[best_index]

    return best_centroids, best_accuracy, np.std(best_fold_accuracies)

def evaluate_robustness(best_model, X_test, y_test, noise_levels=[0.1, 0.2, 0.3, 0.4, 0.5], n_bootstrap=100):
    results = []

    n_samples = len(y_test)

    original_pred = best_model.predict(X_test)
    original_acc = accuracy_score(y_test, original_pred)
    original_f1 = f1_score(y_test, original_pred, average='weighted')
    results.append(('No Noise', original_acc, original_f1))

    bootstrap_original_acc = []
    bootstrap_original_f1 = []

    for _ in range(n_bootstrap):
        indices = np.random.choice(n_samples, n_samples, replace=True)
        X_boot = X_test[indices]
        y_boot = y_test.iloc[indices] if hasattr(y_test, 'iloc') else y_test[indices]

        boot_pred = best_model.predict(X_boot)
        bootstrap_original_acc.append(accuracy_score(y_boot, boot_pred))
        bootstrap_original_f1.append(f1_score(y_boot, boot_pred, average='weighted'))

    p_values_acc = []
    p_values_f1 = []
    bootstrap_acc_all = []
    bootstrap_f1_all = []

    for noise in noise_levels:
        bootstrap_acc = []
        bootstrap_f1 = []

        for _ in range(n_bootstrap):
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_boot = X_test[indices].copy()
            y_boot = y_test.iloc[indices] if hasattr(y_test, 'iloc') else y_test[indices]

            noise_matrix = np.random.normal(0, noise, X_boot.shape)
            X_boot += noise_matrix

            noisy_pred = best_model.predict(X_boot)
            noisy_acc = accuracy_score(y_boot, noisy_pred)
            noisy_f1 = f1_score(y_boot, noisy_pred, average='weighted')

            bootstrap_acc.append(noisy_acc)
            bootstrap_f1.append(noisy_f1)

        mean_acc = np.mean(bootstrap_acc)
        mean_f1 = np.mean(bootstrap_f1)

        t_stat_acc, p_val_acc = stats.ttest_rel(bootstrap_original_acc, bootstrap_acc)
        t_stat_f1, p_val_f1 = stats.ttest_rel(bootstrap_original_f1, bootstrap_f1)

        results.append((f'Noise {noise:.2f}', mean_acc, mean_f1))
        p_values_acc.append(p_val_acc)
        p_values_f1.append(p_val_f1)
        bootstrap_acc_all.append(bootstrap_acc)
        bootstrap_f1_all.append(bootstrap_f1)

    plt.figure(figsize=(12, 7))
    noise_labels = [r[0] for r in results]
    acc_values = [r[1] for r in results]
    f1_values = [r[2] for r in results]

    x = np.arange(len(noise_labels))
    width = 0.35

    plt.subplot(1, 2, 1)
    bars_acc = plt.bar(x - width/2, acc_values, width, label='Accuracy')
    bars_f1 = plt.bar(x + width/2, f1_values, width, label='F1 Score')
    plt.xlabel('Noise Level')
    plt.ylabel('Score')
    plt.title('Model Robustness to Noise')
    plt.xticks(x, noise_labels, rotation=45)
    plt.legend()

    for i in range(len(p_values_acc)):
        if i > 0:
            if p_values_acc[i-1] < 0.001:
                plt.text(i - width/2, acc_values[i] - 0.02, '***', ha='center')
            elif p_values_acc[i-1] < 0.01:
                plt.text(i - width/2, acc_values[i] - 0.02, '**', ha='center')
            elif p_values_acc[i-1] < 0.05:
                plt.text(i - width/2, acc_values[i] - 0.02, '*', ha='center')

            if p_values_f1[i-1] < 0.001:
                plt.text(i + width/2, f1_values[i] - 0.02, '***', ha='center')
            elif p_values_f1[i-1] < 0.01:
                plt.text(i + width/2, f1_values[i] - 0.02, '**', ha='center')
            elif p_values_f1[i-1] < 0.05:
                plt.text(i + width/2, f1_values[i] - 0.02, '*', ha='center')

    plt.subplot(1, 2, 2)
    bars_acc = plt.bar(x - width/2, acc_values, width, label='Accuracy')
    bars_f1 = plt.bar(x + width/2, f1_values, width, label='F1 Score')
    plt.xlabel('Noise Level')
    plt.ylabel('Score')
    plt.title('Zoomed View (0.85-1.0 range) with p-values')
    plt.xticks(x, noise_labels, rotation=45)
    plt.ylim(0.85, 1.0)

    for i in range(len(p_values_acc)):
        if i > 0:
            if p_values_acc[i-1] < 0.001:
                plt.text(i - width/2, acc_values[i] + 0.005, '***', ha='center', fontsize=10)
            elif p_values_acc[i-1] < 0.01:
                plt.text(i - width/2, acc_values[i] + 0.005, '**', ha='center', fontsize=10)
            elif p_values_acc[i-1] < 0.05:
                plt.text(i - width/2, acc_values[i] + 0.005, '*', ha='center', fontsize=10)

            if p_values_f1[i-1] < 0.001:
                plt.text(i + width/2, f1_values[i] + 0.005, '***', ha='center', fontsize=10)
            elif p_values_f1[i-1] < 0.01:
                plt.text(i + width/2, f1_values[i] + 0.005, '**', ha='center', fontsize=10)
            elif p_values_f1[i-1] < 0.05:
                plt.text(i + width/2, f1_values[i] + 0.005, '*', ha='center', fontsize=10)

    for i, (acc, f1) in enumerate(zip(acc_values, f1_values)):
        plt.text(i - width/2, acc - 0.01, f'{acc:.7f}', ha='center', va='bottom', fontsize=8, rotation=90)
        plt.text(i + width/2, f1 - 0.01, f'{f1:.7f}', ha='center', va='bottom', fontsize=8, rotation=90)

    plt.figtext(0.5, 0.01, "* p<0.05, ** p<0.01, *** p<0.001", ha="center", fontsize=10)

    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'rice_noise_robustness.png'))
    plt.close()

    plt.figure(figsize=(14, 8))

    plt.subplot(2, 1, 1)
    boxplot_data_acc = [bootstrap_original_acc] + bootstrap_acc_all
    plt.boxplot(boxplot_data_acc, labels=noise_labels, showfliers=False)
    plt.title('Distribution of Accuracy Scores Across Noise Levels')
    plt.ylabel('Accuracy')
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.subplot(2, 1, 2)
    boxplot_data_f1 = [bootstrap_original_f1] + bootstrap_f1_all
    plt.boxplot(boxplot_data_f1, labels=noise_labels, showfliers=False)
    plt.title('Distribution of F1 Scores Across Noise Levels')
    plt.ylabel('F1 Score')
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'rice_noise_distribution.png'))
    plt.close()

    logging.info("\n===== NOISE ROBUSTNESS RESULTS WITH P-VALUES =====")
    logging.info(f"{'Noise Level':<12} {'Accuracy':<15} {'F1 Score':<15} {'p-val (Acc)':<12} {'p-val (F1)':<12}")

    logging.info(f"{noise_labels[0]:<12} {acc_values[0]:.7f}      {f1_values[0]:.7f}      -            -")

    for i in range(1, len(noise_labels)):
        logging.info(f"{noise_labels[i]:<12} {acc_values[i]:.7f}      {f1_values[i]:.7f}      {p_values_acc[i-1]:.2e}     {p_values_f1[i-1]:.2e}")

    return results, p_values_acc, p_values_f1

def analyze_results():
    acc_mean, acc_std = np.mean(best_accuracies), np.std(best_accuracies)
    prec_mean, prec_std = np.mean(best_precisions), np.std(best_precisions)
    rec_mean, rec_std = np.mean(best_recalls), np.std(best_recalls)
    f1_mean, f1_std = np.mean(best_f1s), np.std(best_f1s)
    time_mean, time_std = np.mean(best_times), np.std(best_times)
    client_time_mean, client_time_std = np.mean(best_client_times), np.std(best_client_times)
    comm_cost_mean, comm_cost_std = np.mean(best_comm_costs), np.std(best_comm_costs)

    logging.info("\n===== BEST CONFIGURATION RESULTS (CLASS-BALANCED CENTROID ALLOCATION) =====")
    logging.info(f"Best number of centroids: {best_centroids}")
    logging.info(f"Number of clients: {args.num_clients}")
    logging.info(f"KNN n_neighbors (k): {args.k}")
    logging.info(f"Optimization Algorithm: Artificial Bee Colony (ABC)")
    logging.info(f"Centroid allocation strategy: Class-balanced (Allocation to each class proportional to its data size)")
    logging.info(f"Accuracy: {acc_mean:.7f} ± {acc_std:.7f}")
    logging.info(f"Precision: {prec_mean:.7f} ± {prec_std:.7f}")
    logging.info(f"Recall: {rec_mean:.7f} ± {rec_std:.7f}")
    logging.info(f"F1 Score: {f1_mean:.7f} ± {f1_std:.7f}")
    logging.info(f"Average time per fold: {time_mean:.7f}s ± {time_std:.7f}s")
    logging.info(f"Average time per client: {client_time_mean:.7f}s ± {client_time_std:.7f}s")
    logging.info(f"Communication cost: {comm_cost_mean:.7f}KB ± {comm_cost_std:.7f}KB")

    if best_y_test is not None:
        unique_labels, counts = np.unique(best_y_test, return_counts=True)
        class_ratios = dict(zip(unique_labels, counts / counts.sum()))

        for label, ratio in class_ratios.items():
            class_name = label_encoder.inverse_transform([label])[0] if hasattr(label_encoder, 'inverse_transform') else f"Class {label}"
            logging.info(f"Class ratio ({class_name}): {ratio:.7f}")

    plt.figure(figsize=(10, 6))
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    values = [acc_mean, prec_mean, rec_mean, f1_mean]
    errors = [acc_std, prec_std, rec_std, f1_std]

    bars = plt.bar(metrics, values, yerr=errors, capsize=10)
    plt.title(f'Performance Metrics with Error Bars (Centroids: {best_centroids}, Clients: {args.num_clients})')
    plt.ylabel('Score')
    plt.ylim(0, 1.1)

    for bar, val, err in zip(bars, values, errors):
        plt.text(bar.get_x() + bar.get_width()/2., val + err + 0.02,
                f'{val:.7f}±{err:.7f}', ha='center', va='bottom', rotation=0)

    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'rice_performance_metrics.png'))
    plt.close()

    if best_y_test is not None and best_predictions is not None:
        plt.figure(figsize=(8, 6))
        cm = confusion_matrix(best_y_test, best_predictions)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f'Confusion Matrix (Centroids: {best_centroids}, Clients: {args.num_clients})')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.savefig(os.path.join(RESULTS_DIR, 'rice_confusion_matrix.png'))
        plt.close()

    plt.figure(figsize=(14, 8))

    plt.subplot(2, 2, 1)
    plt.plot(range(len(all_centroids)), all_accuracies, 'b-', label='Accuracy')
    plt.plot(range(len(all_centroids)), all_f1s, 'g-', label='F1 Score')
    plt.axvline(x=all_centroids.index(best_centroids), color='r', linestyle='--', label=f'Best Centroids={best_centroids}')
    plt.title('Accuracy and F1 Score vs. Iteration')
    plt.xlabel('Iteration')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.subplot(2, 2, 2)
    plt.plot(range(len(all_centroids)), all_precisions, 'm-', label='Precision')
    plt.plot(range(len(all_centroids)), all_recalls, 'c-', label='Recall')
    plt.axvline(x=all_centroids.index(best_centroids), color='r', linestyle='--', label=f'Best Centroids={best_centroids}')
    plt.title('Precision and Recall vs. Iteration')
    plt.xlabel('Iteration')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.subplot(2, 2, 3)
    plt.plot(range(len(all_centroids)), all_times, 'k-', label='Fold Time')
    plt.plot(range(len(all_centroids)), all_client_times, 'y-', label='Client Time')
    plt.axvline(x=all_centroids.index(best_centroids), color='r', linestyle='--', label=f'Best Centroids={best_centroids}')
    plt.title('Execution Time vs. Iteration')
    plt.xlabel('Iteration')
    plt.ylabel('Time (s)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.subplot(2, 2, 4)
    plt.plot(range(len(all_centroids)), all_comm_costs, 'r-', label='Comm Cost')
    plt.axvline(x=all_centroids.index(best_centroids), color='b', linestyle='--', label=f'Best Centroids={best_centroids}')
    plt.title('Communication Cost vs. Iteration')
    plt.xlabel('Iteration')
    plt.ylabel('Communication Cost (KB)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'rice_optimization_progress.png'))
    plt.close()

    return {
        'acc_mean': acc_mean, 'acc_std': acc_std,
        'prec_mean': prec_mean, 'prec_std': prec_std,
        'rec_mean': rec_mean, 'rec_std': rec_std,
        'f1_mean': f1_mean, 'f1_std': f1_std,
        'time_mean': time_mean, 'time_std': time_std,
        'client_time_mean': client_time_mean, 'client_time_std': client_time_std,
        'comm_cost_mean': comm_cost_mean, 'comm_cost_std': comm_cost_std
    }

def main():
    start_time = time.time()

    logging.info(f"Artificial Bee Colony (ABC) optimization starting (with class-balanced centroid allocation)...")
    logging.info(f"Parameter configuration: Number of Clients={args.num_clients}, Max Centroid={max_centroids}, Number of Bees={args.n_bees}, Maximum Iterations={args.n_calls}")

    dimension_bounds = (10, max_centroids)

    best_centroids_abc, best_accuracy_abc, std_dev_abc = abc_algorithm(
        objective_sequential,
        dimension_bounds,
        n_bees=args.n_bees,
        max_iter=args.n_calls
    )

    global best_centroids
    best_centroids = best_centroids_abc

    total_time = time.time() - start_time
    logging.info(f"Optimization completed. Total time: {total_time:.7f} seconds")
    logging.info(f"Best number of centroids: {best_centroids}, Accuracy: {best_accuracy:.7f}, Standard Deviation: {std_dev_abc:.7f}")

    metrics = analyze_results()

    logging.info("Running noise robustness test...")

    best_fold_idx = np.argmax(best_accuracies)
    train_indices = list(kf.split(X))[best_fold_idx][0]
    test_indices = list(kf.split(X))[best_fold_idx][1]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X.iloc[train_indices])
    X_test_scaled = scaler.transform(X.iloc[test_indices])
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]

    client_start_time = time.time()

    num_splits = args.num_clients
    split_train_data = np.array_split(X_train_scaled, num_splits)
    split_train_labels = np.array_split(y_train, num_splits)
    data_counts = np.array([len(data) for data in split_train_data])
    centroids_per_split = distribute_centroids(data_counts, best_centroids)

    client_times = []

    center_labels = {}
    for split_data, split_labels, centroids_count in zip(split_train_data, split_train_labels, centroids_per_split):
        client_iter_start = time.time()

        unique_labels, counts = np.unique(split_labels, return_counts=True)
        label_counts = dict(zip(unique_labels, counts / counts.sum()))
        centroids_per_class = {label: max(1, int(count * centroids_count))
                              for label, count in label_counts.items()}

        total_class_centroids = sum(centroids_per_class.values())
        if total_class_centroids > centroids_count:
            scale_factor = centroids_count / total_class_centroids
            centroids_per_class = {label: max(1, int(count * scale_factor))
                                   for label, count in centroids_per_class.items()}

        logging.info(f"Centroid distribution: {centroids_per_class}")

        for label in unique_labels:
            class_data = split_data[split_labels == label]
            if centroids_per_class[label] > 0 and len(class_data) > 0:
                kmeans = KMeans(n_clusters=min(len(class_data), centroids_per_class[label]), random_state=42)
                kmeans.fit(class_data)
                if label in center_labels:
                    center_labels[label].extend(kmeans.cluster_centers_)
                else:
                    center_labels[label] = list(kmeans.cluster_centers_)

        client_iter_end = time.time()
        client_times.append(client_iter_end - client_iter_start)

    client_end_time = time.time()
    client_total_time = client_end_time - client_start_time
    client_avg_time = np.mean(client_times)

    feature_dim = X.shape[1]
    comm_cost = calculate_communication_cost(centroids_per_split, feature_dim)

    logging.info(f"Client training times: Total={client_total_time:.7f}s, Average={client_avg_time:.7f}s")
    logging.info(f"Per client communication cost: {comm_cost/num_splits:.7f}KB")

    final_centers = [center for sublist in center_labels.values() for center in sublist]
    final_labels = [label for label, centers in center_labels.items() for center in centers]

    best_model = KNeighborsClassifier(n_neighbors=args.k)
    best_model.fit(final_centers, final_labels)

    noise_levels = [0.1, 0.2, 0.3, 0.4, 0.5]
    n_bootstrap = 100  # Bootstrap örnek sayısı

    noise_results, p_values_acc, p_values_f1 = evaluate_robustness(
        best_model, X_test_scaled, y_test, noise_levels, n_bootstrap)

    summary_file = os.path.join(RESULTS_DIR, 'experiment_summary.txt')
    with open(summary_file, 'w') as f:
        f.write(f"===== EXPERIMENT SUMMARY =====\n")
        f.write(f"Experiment Name: {args.experiment_name}\n")
        f.write(f"Number of Clients: {args.num_clients}\n")
        f.write(f"Optimization Algorithm: Artificial Bee Colony (ABC)\n")
        f.write(f"Number of Bees: {args.n_bees}\n")
        f.write(f"Max Iterations: {args.n_calls}\n")
        f.write(f"KNN n_neighbors (k): {args.k}\n")
        f.write(f"Max Centroids: {max_centroids}\n")
        f.write(f"Total Execution Time: {total_time:.2f} seconds\n\n")

        f.write(f"===== BEST RESULTS =====\n")
        f.write(f"Best Centroids: {best_centroids}\n")
        f.write(f"Accuracy: {metrics['acc_mean']:.7f} ± {metrics['acc_std']:.7f}\n")
        f.write(f"Precision: {metrics['prec_mean']:.7f} ± {metrics['prec_std']:.7f}\n")
        f.write(f"Recall: {metrics['rec_mean']:.7f} ± {metrics['rec_std']:.7f}\n")
        f.write(f"F1 Score: {metrics['f1_mean']:.7f} ± {metrics['f1_std']:.7f}\n")
        f.write(f"Client Time: {metrics['client_time_mean']:.7f}s ± {metrics['client_time_std']:.7f}s\n")
        f.write(f"Communication Cost: {metrics['comm_cost_mean']:.7f}KB ± {metrics['comm_cost_std']:.7f}KB\n")

    logging.info(f"Process completed. Results were saved to the {RESULTS_DIR} folder.")
    logging.info(f"Experiment summary: {summary_file}")

if __name__ == "__main__":
    main()

## FlyNNFL

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from skopt import gp_minimize
from skopt.space import Integer
from skopt.utils import use_named_args
from scipy.io import arff
import logging
from joblib import Parallel, delayed
from scipy import stats

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[logging.FileHandler('rice_flynn.log'), logging.StreamHandler()])


class FlyNN:
    def __init__(self, m=2000, s=10, rho=20, gamma=0.5, random_state=42):

        self.m = m
        self.s = s
        self.rho = rho
        self.gamma = gamma
        self.random_state = random_state
        self.M = None
        self.class_fbf = {}

    def _generate_lifting_matrix(self, d):
        np.random.seed(self.random_state)
        M = np.zeros((self.m, d), dtype=np.bool_)

        actual_s = min(self.s, d)

        for i in range(self.m):
            nonzero_indices = np.random.choice(d, actual_s, replace=False)
            M[i, nonzero_indices] = True

        return M

    def _flyhash(self, x):
        if self.M is None:
            raise ValueError("Lifting matrix M has not been initialized")

        if hasattr(x, 'values'):
            x = x.values

        projections = self.M.dot(x)

        actual_rho = min(self.rho, self.m)

        h = np.zeros(self.m, dtype=np.bool_)
        top_indices = np.argsort(projections)[-actual_rho:]
        h[top_indices] = True

        return h

    def fit(self, X, y):
        d = X.shape[1]
        self.M = self._generate_lifting_matrix(d)

        unique_classes = np.unique(y)

        for cls in unique_classes:
            self.class_fbf[cls] = np.ones(self.m, dtype=np.float32)

        for i in range(len(X)):
            if isinstance(X, (pd.DataFrame, pd.Series)):
                x = X.iloc[i]
            else:
                x = X[i]

            if isinstance(y, (pd.DataFrame, pd.Series)):
                label = y.iloc[i]
            else:
                label = y[i]

            h = self._flyhash(x)

            self.class_fbf[label][h] *= self.gamma

        return self

    def predict(self, X):
        if not self.class_fbf:
            raise ValueError("Model has not been trained")

        predictions = np.zeros(len(X), dtype=int)

        for i in range(len(X)):
            if isinstance(X, (pd.DataFrame, pd.Series)):
                x = X.iloc[i]
            else:
                x = X[i]

            h = self._flyhash(x)

            scores = {}
            for cls, fbf in self.class_fbf.items():
                scores[cls] = np.sum(fbf[h])

            predictions[i] = min(scores, key=scores.get)

        return predictions

def train_client_flynn(X_train, y_train, m, s, rho, gamma, random_state):
    client_model = FlyNN(m=m, s=s, rho=rho, gamma=gamma, random_state=random_state)
    client_model.fit(X_train, y_train)
    return client_model.M, client_model.class_fbf

def aggregate_flynn_models(client_models, gamma):
    M = client_models[0][0]

    all_classes = set()
    for _, class_fbf in client_models:
        all_classes.update(class_fbf.keys())

    aggregated_fbf = {}
    for cls in all_classes:
        log_sum = 0
        count = 0

        for _, class_fbf in client_models:
            if cls in class_fbf:
                log_sum += np.log(class_fbf[cls]) / np.log(gamma)
                count += 1

        if count > 0:
            aggregated_fbf[cls] = np.power(gamma, log_sum / count)

    return M, aggregated_fbf

def apply_differential_privacy(class_fbf, epsilon, samples, gamma):
    log_fbf = {}
    for cls, fbf in class_fbf.items():
        log_fbf[cls] = np.log(fbf) / np.log(gamma)

    dp_log_fbf = {}
    for cls, log_counts in log_fbf.items():
        dp_log_fbf[cls] = np.zeros_like(log_counts)

        probabilities = np.exp(log_counts / (4 * samples))
        probabilities = probabilities / np.sum(probabilities)

        selected_indices = np.random.choice(
            len(log_counts),
            size=min(samples, len(log_counts)),
            replace=False,
            p=probabilities
        )

        noise = np.random.laplace(0, 2 * samples / epsilon, size=len(selected_indices))
        for i, idx in enumerate(selected_indices):
            dp_log_fbf[cls][idx] = max(0, log_counts[idx] + noise[i])

    dp_fbf = {}
    for cls, dp_log_counts in dp_log_fbf.items():
        dp_fbf[cls] = np.power(gamma, dp_log_counts)

    return dp_fbf

def federated_train_flynn(split_data, split_labels, m, s, rho, gamma, random_state, is_dp=False, epsilon=1.0, dp_samples=100):
    client_models = []

    for client_data, client_labels in zip(split_data, split_labels):
        M, class_fbf = train_client_flynn(client_data, client_labels, m, s, rho, gamma, random_state)

        if is_dp:
            class_fbf = apply_differential_privacy(class_fbf, epsilon, dp_samples, gamma)

        client_models.append((M, class_fbf))

    M, aggregated_fbf = aggregate_flynn_models(client_models, gamma)

    model = FlyNN(m=m, s=s, rho=rho, gamma=gamma, random_state=random_state)
    model.M = M
    model.class_fbf = aggregated_fbf

    return model

def evaluate_flynn_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return accuracy, precision, recall, f1, y_test, y_pred

def calculate_communication_cost(m, num_classes, feature_dim, clients):
    bytes_per_fbf = m * 4
    bytes_per_client = num_classes * bytes_per_fbf
    total_bytes = clients * bytes_per_client

    return total_bytes / 1024

space = [
    Integer(2000, 10000, name='m'),
    Integer(5, 50, name='s'),
    Integer(10, 100, name='rho'),
]

@use_named_args(space)
def objective_flynn(m, s, rho):
    logging.info(f"Evaluating FlyNN hyperparameters: m={m}, s={s}, rho={rho}")

    gamma = 0.5
    num_splits = 10

    fold_accuracies = []
    fold_precisions = []
    fold_recalls = []
    fold_f1s = []
    fold_times = []
    fold_client_times = []
    fold_comm_costs = []

    for train_index, test_index in kf.split(X):
        fold_start_time = time.time()

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X.iloc[train_index])
        X_test_scaled = scaler.transform(X.iloc[test_index])
        y_train, y_test = y[train_index], y[test_index]

        X_train_scaled = np.array(X_train_scaled)
        X_test_scaled = np.array(X_test_scaled)
        y_train = np.array(y_train)
        y_test = np.array(y_test)

        split_train_data = np.array_split(X_train_scaled, num_splits)
        split_train_labels = np.array_split(y_train, num_splits)

        client_times = []

        client_start_time = time.time()

        flynn_model = federated_train_flynn(
            split_train_data,
            split_train_labels,
            m, s, rho, gamma,
            random_state=42,
            is_dp=False
        )

        client_end_time = time.time()
        client_times.append(client_end_time - client_start_time)

        accuracy, precision, recall, f1, _, _ = evaluate_flynn_model(flynn_model, X_test_scaled, y_test)

        fold_end_time = time.time()
        fold_exec_time = fold_end_time - fold_start_time

        num_classes = len(np.unique(y_train))
        feature_dim = X.shape[1]
        comm_cost = calculate_communication_cost(m, num_classes, feature_dim, num_splits)

        fold_accuracies.append(accuracy)
        fold_precisions.append(precision)
        fold_recalls.append(recall)
        fold_f1s.append(f1)
        fold_times.append(fold_exec_time)
        fold_client_times.append(np.mean(client_times))
        fold_comm_costs.append(comm_cost)

    mean_accuracy = np.mean(fold_accuracies)
    mean_precision = np.mean(fold_precisions)
    mean_recall = np.mean(fold_recalls)
    mean_f1 = np.mean(fold_f1s)
    mean_time = np.mean(fold_times)
    mean_client_time = np.mean(fold_client_times)
    mean_comm_cost = np.mean(fold_comm_costs)

    all_hyperparams.append((m, s, rho))
    all_accuracies.append(mean_accuracy)
    all_precisions.append(mean_precision)
    all_recalls.append(mean_recall)
    all_f1s.append(mean_f1)
    all_times.append(mean_time)
    all_client_times.append(mean_client_time)
    all_comm_costs.append(mean_comm_cost)

    global best_accuracy, best_params, best_accuracies, best_precisions, best_recalls
    global best_f1s, best_times, best_client_times, best_comm_costs, best_y_test, best_predictions

    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_params = (m, s, rho)
        best_accuracies = fold_accuracies
        best_precisions = fold_precisions
        best_recalls = fold_recalls
        best_f1s = fold_f1s
        best_times = fold_times
        best_client_times = fold_client_times
        best_comm_costs = fold_comm_costs

    logging.info(f"FlyNN: m={m}, s={s}, rho={rho}, Acc: {mean_accuracy:.7f}, Prec: {mean_precision:.7f}, " +
                f"Rec: {mean_recall:.7f}, F1: {mean_f1:.7f}, Time: {mean_time:.2f}s, " +
                f"Client Time: {mean_client_time:.7f}s, Comm: {mean_comm_cost:.2f}KB")

    return -mean_accuracy


def evaluate_flynn_robustness(best_model, X_test, y_test, noise_levels=[0.1, 0.2, 0.3, 0.4, 0.5], n_bootstrap=100):
    results = []

    X_test = np.array(X_test)
    y_test = np.array(y_test)

    n_samples = len(y_test)

    original_pred = best_model.predict(X_test)
    original_acc = accuracy_score(y_test, original_pred)
    original_f1 = f1_score(y_test, original_pred, average='weighted')
    results.append(('No Noise', original_acc, original_f1))

    bootstrap_original_acc = []
    bootstrap_original_f1 = []

    for _ in range(n_bootstrap):
        indices = np.random.choice(n_samples, n_samples, replace=True)
        X_boot = X_test[indices]
        y_boot = y_test[indices]

        boot_pred = best_model.predict(X_boot)
        bootstrap_original_acc.append(accuracy_score(y_boot, boot_pred))
        bootstrap_original_f1.append(f1_score(y_boot, boot_pred, average='weighted'))

    p_values_acc = []
    p_values_f1 = []
    bootstrap_acc_all = []
    bootstrap_f1_all = []

    for noise in noise_levels:
        bootstrap_acc = []
        bootstrap_f1 = []

        for _ in range(n_bootstrap):
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_boot = X_test[indices].copy()
            y_boot = y_test[indices]

            noise_matrix = np.random.normal(0, noise, X_boot.shape)
            X_boot += noise_matrix

            noisy_pred = best_model.predict(X_boot)
            noisy_acc = accuracy_score(y_boot, noisy_pred)
            noisy_f1 = f1_score(y_boot, noisy_pred, average='weighted')

            bootstrap_acc.append(noisy_acc)
            bootstrap_f1.append(noisy_f1)

        mean_acc = np.mean(bootstrap_acc)
        mean_f1 = np.mean(bootstrap_f1)

        t_stat_acc, p_val_acc = stats.ttest_rel(bootstrap_original_acc, bootstrap_acc)
        t_stat_f1, p_val_f1 = stats.ttest_rel(bootstrap_original_f1, bootstrap_f1)

        results.append((f'Noise {noise:.2f}', mean_acc, mean_f1))
        p_values_acc.append(p_val_acc)
        p_values_f1.append(p_val_f1)
        bootstrap_acc_all.append(bootstrap_acc)
        bootstrap_f1_all.append(bootstrap_f1)

    visualize_robustness_results(results, p_values_acc, p_values_f1, bootstrap_original_acc, bootstrap_original_f1, bootstrap_acc_all, bootstrap_f1_all)

    return results, p_values_acc, p_values_f1

def visualize_robustness_results(results, p_values_acc, p_values_f1, bootstrap_original_acc, bootstrap_original_f1, bootstrap_acc_all, bootstrap_f1_all):
    plt.figure(figsize=(12, 7))
    noise_labels = [r[0] for r in results]
    acc_values = [r[1] for r in results]
    f1_values = [r[2] for r in results]

    x = np.arange(len(noise_labels))
    width = 0.35

    plt.subplot(1, 2, 1)
    bars_acc = plt.bar(x - width/2, acc_values, width, label='Accuracy')
    bars_f1 = plt.bar(x + width/2, f1_values, width, label='F1 Score')
    plt.xlabel('Noise Level')
    plt.ylabel('Score')
    plt.title('FlyNN Model Robustness to Noise')
    plt.xticks(x, noise_labels, rotation=45)
    plt.legend()

    for i in range(len(p_values_acc)):
        if i > 0:
            if p_values_acc[i-1] < 0.001:
                plt.text(i - width/2, acc_values[i] - 0.02, '***', ha='center')
            elif p_values_acc[i-1] < 0.01:
                plt.text(i - width/2, acc_values[i] - 0.02, '**', ha='center')
            elif p_values_acc[i-1] < 0.05:
                plt.text(i - width/2, acc_values[i] - 0.02, '*', ha='center')

            if p_values_f1[i-1] < 0.001:
                plt.text(i + width/2, f1_values[i] - 0.02, '***', ha='center')
            elif p_values_f1[i-1] < 0.01:
                plt.text(i + width/2, f1_values[i] - 0.02, '**', ha='center')
            elif p_values_f1[i-1] < 0.05:
                plt.text(i + width/2, f1_values[i] - 0.02, '*', ha='center')

    plt.subplot(1, 2, 2)
    bars_acc = plt.bar(x - width/2, acc_values, width, label='Accuracy')
    bars_f1 = plt.bar(x + width/2, f1_values, width, label='F1 Score')
    plt.xlabel('Noise Level')
    plt.ylabel('Score')
    plt.title('Zoomed View (0.85-1.0 range) with p-values')
    plt.xticks(x, noise_labels, rotation=45)
    plt.ylim(0.85, 1.0)

    for i in range(len(p_values_acc)):
        if i > 0:
            if p_values_acc[i-1] < 0.001:
                plt.text(i - width/2, acc_values[i] + 0.005, '***', ha='center', fontsize=10)
            elif p_values_acc[i-1] < 0.01:
                plt.text(i - width/2, acc_values[i] + 0.005, '**', ha='center', fontsize=10)
            elif p_values_acc[i-1] < 0.05:
                plt.text(i - width/2, acc_values[i] + 0.005, '*', ha='center', fontsize=10)

            if p_values_f1[i-1] < 0.001:
                plt.text(i + width/2, f1_values[i] + 0.005, '***', ha='center', fontsize=10)
            elif p_values_f1[i-1] < 0.01:
                plt.text(i + width/2, f1_values[i] + 0.005, '**', ha='center', fontsize=10)
            elif p_values_f1[i-1] < 0.05:
                plt.text(i + width/2, f1_values[i] + 0.005, '*', ha='center', fontsize=10)

    for i, (acc, f1) in enumerate(zip(acc_values, f1_values)):
        plt.text(i - width/2, acc - 0.01, f'{acc:.7f}', ha='center', va='bottom', fontsize=8, rotation=90)
        plt.text(i + width/2, f1 - 0.01, f'{f1:.7f}', ha='center', va='bottom', fontsize=8, rotation=90)

    plt.figtext(0.5, 0.01, "* p<0.05, ** p<0.01, *** p<0.001", ha="center", fontsize=10)
    plt.tight_layout()
    plt.savefig('flynn_noise_robustness.png')
    plt.close()

    plt.figure(figsize=(14, 8))

    plt.subplot(2, 1, 1)
    boxplot_data_acc = [bootstrap_original_acc] + bootstrap_acc_all
    plt.boxplot(boxplot_data_acc, labels=noise_labels, showfliers=False)
    plt.title('Distribution of Accuracy Scores Across Noise Levels (FlyNN)')
    plt.ylabel('Accuracy')
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.subplot(2, 1, 2)
    boxplot_data_f1 = [bootstrap_original_f1] + bootstrap_f1_all
    plt.boxplot(boxplot_data_f1, labels=noise_labels, showfliers=False)
    plt.title('Distribution of F1 Scores Across Noise Levels (FlyNN)')
    plt.ylabel('F1 Score')
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig('flynn_noise_distribution.png')
    plt.close()

def analyze_flynn_results():
    acc_mean, acc_std = np.mean(best_accuracies), np.std(best_accuracies)
    prec_mean, prec_std = np.mean(best_precisions), np.std(best_precisions)
    rec_mean, rec_std = np.mean(best_recalls), np.std(best_recalls)
    f1_mean, f1_std = np.mean(best_f1s), np.std(best_f1s)
    time_mean, time_std = np.mean(best_times), np.std(best_times)
    client_time_mean, client_time_std = np.mean(best_client_times), np.std(best_client_times)
    comm_cost_mean, comm_cost_std = np.mean(best_comm_costs), np.std(best_comm_costs)

    logging.info("\n===== FLYNN: BEST CONFIGURATION RESULTS =====")
    logging.info(f"Best parameters: m={best_params[0]}, s={best_params[1]}, rho={best_params[2]}")
    logging.info(f"Accuracy: {acc_mean:.7f} ± {acc_std:.7f}")
    logging.info(f"Precision: {prec_mean:.7f} ± {prec_std:.7f}")
    logging.info(f"Recall: {rec_mean:.7f} ± {rec_std:.7f}")
    logging.info(f"F1 Score: {f1_mean:.7f} ± {f1_std:.7f}")
    logging.info(f"Average time per fold: {time_mean:.7f}s ± {time_std:.7f}s")
    logging.info(f"Average time per client: {client_time_mean:.7f}s ± {client_time_std:.7f}s")
    logging.info(f"Communication cost: {comm_cost_mean:.2f}KB ± {comm_cost_std:.2f}KB")

    plt.figure(figsize=(10, 6))
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    values = [acc_mean, prec_mean, rec_mean, f1_mean]
    errors = [acc_std, prec_std, rec_std, f1_std]

    bars = plt.bar(metrics, values, yerr=errors, capsize=10)
    plt.title(f'FlyNN Performance Metrics (m={best_params[0]}, s={best_params[1]}, rho={best_params[2]})')
    plt.ylabel('Score')
    plt.ylim(0, 1.1)

    for bar, val, err in zip(bars, values, errors):
        plt.text(bar.get_x() + bar.get_width()/2., val + err + 0.02,
                f'{val:.7f}±{err:.7f}', ha='center', va='bottom', rotation=0)

    plt.tight_layout()
    plt.savefig('flynn_performance_metrics.png')
    plt.close()

    if best_y_test is not None and best_predictions is not None:
        plt.figure(figsize=(8, 6))
        cm = confusion_matrix(best_y_test, best_predictions)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix for Best FlyNN Configuration')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.savefig('flynn_confusion_matrix.png')
        plt.close()

def main_flynn():
    global X, y, kf
    global best_accuracy, best_params, best_accuracies, best_precisions, best_recalls
    global best_f1s, best_times, best_client_times, best_comm_costs, best_y_test, best_predictions
    global all_hyperparams, all_accuracies, all_precisions, all_recalls, all_f1s, all_times, all_client_times, all_comm_costs

    data_path = 'rice/Rice_Cammeo_Osmancik.arff'
    data, meta = arff.loadarff(data_path)
    data_df = pd.DataFrame(data)

    data_df['Class'] = data_df['Class'].apply(lambda x: x.decode('utf-8'))

    label_encoder = LabelEncoder()
    data_df['Class'] = label_encoder.fit_transform(data_df['Class'])

    X = data_df.drop(columns=['Class'])
    y = data_df['Class']

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    all_hyperparams = []
    all_accuracies = []
    all_precisions = []
    all_recalls = []
    all_f1s = []
    all_times = []
    all_client_times = []
    all_comm_costs = []

    best_accuracy = 0
    best_params = None
    best_accuracies = []
    best_precisions = []
    best_recalls = []
    best_f1s = []
    best_times = []
    best_client_times = []
    best_comm_costs = []
    best_y_test = None
    best_predictions = None

    start_time = time.time()

    logging.info("FlyNN Bayesian optimization starting...")
    res_gp = gp_minimize(objective_flynn, space, n_calls=50, random_state=0)

    total_time = time.time() - start_time
    logging.info(f"FlyNN optimization completed. Total time: {total_time:.2f} seconds")

    m_best, s_best, rho_best = best_params
    logging.info(f"Best FlyNN parameters: m={m_best}, s={s_best}, rho={rho_best}")

    analyze_flynn_results()

    logging.info("FlyNN: Running noise robustness test...")

    best_fold_idx = np.argmax(best_accuracies)
    train_indices = list(kf.split(X))[best_fold_idx][0]
    test_indices = list(kf.split(X))[best_fold_idx][1]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X.iloc[train_indices])
    X_test_scaled = scaler.transform(X.iloc[test_indices])
    y_train, y_test = y[train_indices], y[test_indices]

    X_train_scaled = np.array(X_train_scaled)
    X_test_scaled = np.array(X_test_scaled)
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    num_splits = 10
    split_train_data = np.array_split(X_train_scaled, num_splits)
    split_train_labels = np.array_split(y_train, num_splits)

    gamma = 0.5
    best_flynn_model = federated_train_flynn(
        split_train_data,
        split_train_labels,
        m_best, s_best, rho_best, gamma,
        random_state=42,
        is_dp=False
    )

    best_y_test = y_test
    best_predictions = best_flynn_model.predict(X_test_scaled)

    noise_levels = [0.1, 0.2, 0.3, 0.4, 0.5]
    n_bootstrap = 100

    noise_results, p_values_acc, p_values_f1 = evaluate_flynn_robustness(
        best_flynn_model, X_test_scaled, y_test, noise_levels, n_bootstrap)

    logging.info("===== NOISE ROBUSTNESS RESULTS WITH P-VALUES =====")
    logging.info(f"{'Noise Level':<12} {'Accuracy':<12} {'F1 Score':<12} {'p-val (Acc)':<12} {'p-val (F1)':<12}")

    logging.info(f"{noise_results[0][0]:<12} {noise_results[0][1]:.7f}      {noise_results[0][2]:.7f}      -            -")

    for i in range(1, len(noise_results)):
        logging.info(f"{noise_results[i][0]:<12} {noise_results[i][1]:.7f}      {noise_results[i][2]:.7f}      {p_values_acc[i-1]:.2e}     {p_values_f1[i-1]:.2e}")

    logging.info("FlyNN process completed.")

    return best_params, best_accuracies, best_precisions, best_recalls, best_f1s, best_times, best_client_times, best_comm_costs

if __name__ == "__main__":
    main_flynn()